# Setting up the environment

We'll load the needed libraries:


In [1]:
library(styler)
styler::style_file(path = "Calculations.Rmd")

In [1]:
options(repr.matrix.max.rows = 100, repr.matrix.max.cols = 300)
options(repr.plot.width = 20, repr.plot.height = 15)
options(width = 300)

numcores <- 56

library(tidyverse)
library(data.table)
library(fst)
library(comorbidity)
library(reshape)
library(dtplyr)
library(haven)
library(vroom)
library(dplyr)
`%!in%` <- Negate(`%in%`)

setDTthreads(numcores)

── Attaching packages ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose



Attaching package: ‘reshape’


The following object is masked from ‘package:data.table’:

    melt


The following object is masked from ‘package:dplyr’:

    rename

# Codes

First, we will add codes from ICD and Medicare:primary_care_specialty_codes

In [2]:
# diagnosis codes

office_visit_codes <- c(
  "99201", "99202", "99203", "99204", "99205", "99211", "99212", "99213", "99214",
  "99215"
)

IHD_icd_9_codes <- c(410, 411, 412, 413, 414)
IHD_icd_10_codes <- c("I20", "I21", "I22", "I23", "I24", "I25")

non_us_state_codes <- c(40, 54, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 97, 98, 99)

primary_care_specialty_codes <- c("01", "08", "11", "38")

# http://www.icd9data.com/2015/Volume1/390-459/401-405/default.htm
# https://www.icd10data.com/ICD10CM/Codes/I00-I99/I10-I16
hypertension_icd_9_codes <- c("401", "402", "403", "404", "405")
hypertension_icd_10_codes <- c("I10", "I11", "I12", "I13", "I15", "I16")

# http://www.icd9data.com/2014/Volume1/290-319/295-299/296/default.htm
# https://www.icd10data.com/ICD10CM/Codes/F01-F99/F30-F39
depression_icd_9_codes <- c("2962", "2963")
depression_icd_10_codes <- c("F32", "F33")

# http://www.icd9data.com/2015/Volume1/240-279/249-259/default.htm
# https://www.icd10data.com/ICD10CM/Codes/E00-E89/E08-E13
diabetes_icd_9_codes <- c("250")
diabetes_icd_10_codes <- c("E08", "E09", "E10", "E11", "E13")

# http://www.icd9data.com/2014/Volume1/710-739/710-719/714/default.htm
# https://www.icd10data.com/ICD10CM/Codes/M00-M99/M05-M14
arthritis_icd_9_codes <- c("714")
arthritis_icd_10_codes <- c("M05", "M06", "M07", "M08", "M09", "M10", "M11", "M12", "M13", "M14")


race_codes <- data.frame(
  race_code = seq(0, 6),
  race = c("Unknown", "White", "Black", "Other", "Asian", "Hispanic", "North American Native")
)

sex_codes <- data.frame(
  sex_code = seq(0, 2),
  sex = c("Unknown", "Male", "Female")
)

# Patient level calculations

## Yearly Calculators

These are the main functions that calculate yearly expenditures for patients and their corresponding physicians.\

### Fixing the MBSF data

In [10]:
mbsf_2013 <- vroom(file = "/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/den_saf_lds_5_2013.csv", num_threads = numcores) %>% as.data.table()
mbsf_2014 <- vroom(file = "/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/den_saf_lds_5_2014.csv", num_threads = numcores) %>% as.data.table()
mbsf_2015 <- vroom(file = "/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/den_saf_lds_5_2015.csv", num_threads = numcores) %>% as.data.table()
mbsf_2016 <- vroom(file = "/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/mbsf_lds_5_2016.csv", num_threads = numcores) %>% as.data.table()
mbsf_2017 <- vroom(file = "/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/mbsf_lds_5_2017.csv", num_threads = numcores) %>% as.data.table()
mbsf_2018 <- vroom(file = "/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/mbsf_lds_5_2018.csv", num_threads = numcores) %>% as.data.table()
mbsf_2019 <- vroom(file = "/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/mbsf_lds_5_2019.csv", num_threads = numcores) %>% as.data.table()
mbsf_2020 <- vroom(file = "/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/mbsf_lds_5_2020.csv", num_threads = numcores) %>% as.data.table()

New names:
• `1` -> `1...5`
• `1` -> `1...7`
• `0` -> `0...8`
• `0` -> `0...9`
• `0` -> `0...11`
• `0` -> `0...12`
• `3` -> `3...13`
• `3` -> `3...14`
• `3` -> `3...15`
• `3` -> `3...16`
• `3` -> `3...17`
• `3` -> `3...18`
• `3` -> `3...19`
• `3` -> `3...20`
• `3` -> `3...21`
• `3` -> `3...22`
• `3` -> `3...23`
• `3` -> `3...24`
• `C` -> `C...25`
• `C` -> `C...26`
• `C` -> `C...27`
• `C` -> `C...28`
• `C` -> `C...29`
• `C` -> `C...30`
• `C` -> `C...31`
• `C` -> `C...32`
• `C` -> `C...33`
• `C` -> `C...34`
• `C` -> `C...35`
• `C` -> `C...36`
• `12` -> `12...37`
• `12` -> `12...38`
• `12` -> `12...39`
• `0` -> `0...40`
• `` -> `...41`
• `` -> `...42`
Rows: 2758309 Columns: 43
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (29): 000000000, 05, 400, 0...9, 3...13, 3...14, 3...15, 3...16, 3...17, 3...18, 3...19, 3...20, 3...21, 3...22, 3...23, 3...24, C...25, C...26, C...27, C...

In [11]:
mbsf_colnames_2013_2015 <- c("DESY_SORT_KEY", "STATE_CODE", "COUNTY_CODE", "SEX_CODE", "RACE_CODE", "AGE", "ORIG_REASON_FOR_ENTITLEMENT", "CURR_REASON_FOR_ENTITLEMENT", "ESRD_INDICATOR", "MEDICARE_STATUS_CD", "PART_A_TERMINATION_CODE", "PART_B_TERMINATION_CODE", "ENTITLEMENT_BUY_IN_IND01", "ENTITLEMENT_BUY_IN_IND02", "ENTITLEMENT_BUY_IN_IND03", "ENTITLEMENT_BUY_IN_IND04", "ENTITLEMENT_BUY_IN_IND05", "ENTITLEMENT_BUY_IN_IND06", "ENTITLEMENT_BUY_IN_IND07", "ENTITLEMENT_BUY_IN_IND08", "ENTITLEMENT_BUY_IN_IND09", "ENTITLEMENT_BUY_IN_IND10", "ENTITLEMENT_BUY_IN_IND11", "ENTITLEMENT_BUY_IN_IND12", "HMO_INDICATOR01", "HMO_INDICATOR02", "HMO_INDICATOR03", "HMO_INDICATOR04", "HMO_INDICATOR05", "HMO_INDICATOR06", "HMO_INDICATOR07", "HMO_INDICATOR08", "HMO_INDICATOR09", "HMO_INDICATOR10", "HMO_INDICATOR11", "HMO_INDICATOR12", "HI_COVERAGE", "SMI_COVERAGE", "HMO_COVERAGE", "STATE_BUY_IN_COVERAGE", "VALID_DATE_OF_DEATH_SWITCH", "DATE_OF_DEATH", "REFERENCE_YEAR")

mbsf_colnames_2016_2020 <- c("DESY_SORT_KEY", "REFERENCE_YEAR", "SAMPLE_GROUP", "STATE_CODE", "COUNTY_CODE", "STATE_CNTY_FIPS_CD_01", "STATE_CNTY_FIPS_CD_02", "STATE_CNTY_FIPS_CD_03", "STATE_CNTY_FIPS_CD_04", "STATE_CNTY_FIPS_CD_05", "STATE_CNTY_FIPS_CD_06", "STATE_CNTY_FIPS_CD_07", "STATE_CNTY_FIPS_CD_08", "STATE_CNTY_FIPS_CD_09", "STATE_CNTY_FIPS_CD_10", "STATE_CNTY_FIPS_CD_11", "STATE_CNTY_FIPS_CD_12", "SEX_CODE", "RACE_CODE", "AGE", "ORIG_REASON_FOR_ENTITLEMENT", "CURR_REASON_FOR_ENTITLEMENT", "ESRD_INDICATOR", "MDCR_STATUS_CODE_01", "MDCR_STATUS_CODE_02", "MDCR_STATUS_CODE_03", "MDCR_STATUS_CODE_04", "MDCR_STATUS_CODE_05", "MDCR_STATUS_CODE_06", "MDCR_STATUS_CODE_07", "MDCR_STATUS_CODE_08", "MDCR_STATUS_CODE_09", "MDCR_STATUS_CODE_10", "MDCR_STATUS_CODE_11", "MDCR_STATUS_CODE_12", "PART_A_TERMINATION_CODE", "PART_B_TERMINATION_CODE", "ENTITLEMENT_BUY_IN_IND01", "ENTITLEMENT_BUY_IN_IND02", "ENTITLEMENT_BUY_IN_IND03", "ENTITLEMENT_BUY_IN_IND04", "ENTITLEMENT_BUY_IN_IND05", "ENTITLEMENT_BUY_IN_IND06", "ENTITLEMENT_BUY_IN_IND07", "ENTITLEMENT_BUY_IN_IND08", "ENTITLEMENT_BUY_IN_IND09", "ENTITLEMENT_BUY_IN_IND10", "ENTITLEMENT_BUY_IN_IND11", "ENTITLEMENT_BUY_IN_IND12", "HMO_INDICATOR01", "HMO_INDICATOR02", "HMO_INDICATOR03", "HMO_INDICATOR04", "HMO_INDICATOR05", "HMO_INDICATOR06", "HMO_INDICATOR07", "HMO_INDICATOR08", "HMO_INDICATOR09", "HMO_INDICATOR10", "HMO_INDICATOR11", "HMO_INDICATOR12", "HI_COVERAGE", "SMI_COVERAGE", "HMO_COVERAGE", "STATE_BUY_IN_COVERAGE", "VALID_DATE_OF_DEATH_SWITCH", "DATE_OF_DEATH", "DUAL_STUS_CD_01", "DUAL_STUS_CD_02", "DUAL_STUS_CD_03", "DUAL_STUS_CD_04", "DUAL_STUS_CD_05", "DUAL_STUS_CD_06", "DUAL_STUS_CD_07", "DUAL_STUS_CD_08", "DUAL_STUS_CD_09", "DUAL_STUS_CD_10", "DUAL_STUS_CD_11", "DUAL_STUS_CD_12")


colnames(mbsf_2013) <- mbsf_colnames_2013_2015
colnames(mbsf_2014) <- mbsf_colnames_2013_2015
colnames(mbsf_2015) <- mbsf_colnames_2013_2015
colnames(mbsf_2016) <- mbsf_colnames_2016_2020
colnames(mbsf_2017) <- mbsf_colnames_2016_2020
colnames(mbsf_2018) <- mbsf_colnames_2016_2020
colnames(mbsf_2019) <- mbsf_colnames_2016_2020
colnames(mbsf_2020) <- mbsf_colnames_2016_2020

In [35]:
mbsf_needed_cols <- c(
  "DESY_SORT_KEY",
  "REFERENCE_YEAR",
  "STATE_CODE",
  "COUNTY_CODE",
  "SEX_CODE",
  "RACE_CODE",
  "AGE",
  "ORIG_REASON_FOR_ENTITLEMENT",
  "CURR_REASON_FOR_ENTITLEMENT",
  "ENTITLEMENT_BUY_IN_IND01",
  "ENTITLEMENT_BUY_IN_IND02",
  "ENTITLEMENT_BUY_IN_IND03",
  "ENTITLEMENT_BUY_IN_IND04",
  "ENTITLEMENT_BUY_IN_IND05",
  "ENTITLEMENT_BUY_IN_IND06",
  "ENTITLEMENT_BUY_IN_IND07",
  "ENTITLEMENT_BUY_IN_IND08",
  "ENTITLEMENT_BUY_IN_IND09",
  "ENTITLEMENT_BUY_IN_IND10",
  "ENTITLEMENT_BUY_IN_IND11",
  "ENTITLEMENT_BUY_IN_IND12",
  "HMO_INDICATOR01",
  "HMO_INDICATOR02",
  "HMO_INDICATOR03",
  "HMO_INDICATOR04",
  "HMO_INDICATOR05",
  "HMO_INDICATOR06",
  "HMO_INDICATOR07",
  "HMO_INDICATOR08",
  "HMO_INDICATOR09",
  "HMO_INDICATOR10",
  "HMO_INDICATOR11",
  "HMO_INDICATOR12",
  "VALID_DATE_OF_DEATH_SWITCH",
  "DATE_OF_DEATH"
)

mbsf_data <- list(mbsf_2013, mbsf_2014, mbsf_2015, mbsf_2016, mbsf_2017, mbsf_2018, mbsf_2019, mbsf_2020)

for (a in 1:length(mbsf_data)) {
  mbsf_data[[a]] <- mbsf_data[[a]][, ..mbsf_needed_cols]
}

# rename last n columns in a dataset
rename_last <- function(data, how_many, new_names) {
  total_cols <- ncol(data)
  setnames(data, (total_cols - how_many + 1):(total_cols), new_names)
}


for (a in 1:length(mbsf_data)) {
  mbsf_data[[a]][, year := 2012 + a]
}



mbsf_data <- reduce(mbsf_data, function(x, y) {
  rbind(x, y)
}) %>% as.data.table()

In [37]:
head(mbsf_data)

DESY_SORT_KEY,REFERENCE_YEAR,STATE_CODE,COUNTY_CODE,SEX_CODE,RACE_CODE,AGE,ORIG_REASON_FOR_ENTITLEMENT,CURR_REASON_FOR_ENTITLEMENT,ENTITLEMENT_BUY_IN_IND01,ENTITLEMENT_BUY_IN_IND02,ENTITLEMENT_BUY_IN_IND03,ENTITLEMENT_BUY_IN_IND04,ENTITLEMENT_BUY_IN_IND05,ENTITLEMENT_BUY_IN_IND06,ENTITLEMENT_BUY_IN_IND07,ENTITLEMENT_BUY_IN_IND08,ENTITLEMENT_BUY_IN_IND09,ENTITLEMENT_BUY_IN_IND10,ENTITLEMENT_BUY_IN_IND11,ENTITLEMENT_BUY_IN_IND12,HMO_INDICATOR01,HMO_INDICATOR02,HMO_INDICATOR03,HMO_INDICATOR04,HMO_INDICATOR05,HMO_INDICATOR06,HMO_INDICATOR07,HMO_INDICATOR08,HMO_INDICATOR09,HMO_INDICATOR10,HMO_INDICATOR11,HMO_INDICATOR12,VALID_DATE_OF_DEATH_SWITCH,DATE_OF_DEATH,year
<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
000000000,13,45,910,2,1,75,0,0,C,C,C,C,C,C,C,C,C,C,C,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2013
000000000,13,22,170,2,1,71,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2013
000000000,13,33,420,2,1,93,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2013
000000000,13,49,801,2,1,71,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2013
000000000,13,33,400,2,1,75,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2013
000000000,13,10,510,1,1,70,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2013


In [38]:
mbsf_data_death_collapsed <- mbsf_data[!is.na(DATE_OF_DEATH)]

mbsf_data_death_collapsed$date_died <- mbsf_data_death_collapsed$DATE_OF_DEATH

mbsf_data <-
  left_join(mbsf_data,
    mbsf_data_death_collapsed[, .(
      DESY_SORT_KEY,
      date_died
    )],
    by = ("DESY_SORT_KEY")
  ) %>% as.data.table()

mbsf_data_death_collapsed <- mbsf_data[!is.na(VALID_DATE_OF_DEATH_SWITCH)]

mbsf_data_death_collapsed$date_died_valid <- mbsf_data_death_collapsed$VALID_DATE_OF_DEATH_SWITCH

mbsf_data <-
  left_join(mbsf_data,
    mbsf_data_death_collapsed[, .(
      DESY_SORT_KEY,
      date_died_valid
    )],
    by = ("DESY_SORT_KEY")
  ) %>% as.data.table()


head(mbsf_data)

DESY_SORT_KEY,REFERENCE_YEAR,STATE_CODE,COUNTY_CODE,SEX_CODE,RACE_CODE,AGE,ORIG_REASON_FOR_ENTITLEMENT,CURR_REASON_FOR_ENTITLEMENT,ENTITLEMENT_BUY_IN_IND01,ENTITLEMENT_BUY_IN_IND02,ENTITLEMENT_BUY_IN_IND03,ENTITLEMENT_BUY_IN_IND04,ENTITLEMENT_BUY_IN_IND05,ENTITLEMENT_BUY_IN_IND06,ENTITLEMENT_BUY_IN_IND07,ENTITLEMENT_BUY_IN_IND08,ENTITLEMENT_BUY_IN_IND09,ENTITLEMENT_BUY_IN_IND10,ENTITLEMENT_BUY_IN_IND11,ENTITLEMENT_BUY_IN_IND12,HMO_INDICATOR01,HMO_INDICATOR02,HMO_INDICATOR03,HMO_INDICATOR04,HMO_INDICATOR05,HMO_INDICATOR06,HMO_INDICATOR07,HMO_INDICATOR08,HMO_INDICATOR09,HMO_INDICATOR10,HMO_INDICATOR11,HMO_INDICATOR12,VALID_DATE_OF_DEATH_SWITCH,DATE_OF_DEATH,year,date_died
<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
000000000,13,45,910,2,1,75,0,0,C,C,C,C,C,C,C,C,C,C,C,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2013,20131017
000000000,13,45,910,2,1,75,0,0,C,C,C,C,C,C,C,C,C,C,C,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2013,20150627
000000000,13,22,170,2,1,71,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2013,20131017
000000000,13,22,170,2,1,71,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2013,20150627
000000000,13,33,420,2,1,93,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2013,20131017
000000000,13,33,420,2,1,93,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2013,20150627


In [43]:
write_fst(mbsf_data, "/work/postresearch/Shared/Projects/Farbod/CaseMix/mbsf_data_long.fst")

### Read data



In [190]:
carrier_data_all_years <- read_fst(
  "/work/postresearch/Shared/Projects/Farbod/carrier_data_all_years.fst",
  as.data.table = T
)
# mbsf_data = read_fst(
#  "/work/postresearch/Shared/Projects/Farbod/CaseMix/mbsf_data_long.fst", as.data.table = T)
# mbsf_data[,DESY_SORT_KEY := as.integer(DESY_SORT_KEY)]

In [191]:
head(carrier_data_all_years)
head(mbsf_data)

DESY_SORT_KEY,CLAIM_NO,LINE_NUM,CLM_THRU_DT,LINE_PLACE_OF_SRVC_CD,HCPCS_CD,LINE_ICD_DGNS_VRSN_CD,LINE_ICD_DGNS_CD,LINE_ALOWD_CHRG_AMT,PRF_PHYSN_NPI,PRVDR_SPCLTY,PRVDR_STATE_CD,date,year,month_year
<int>,<int>,<int>,<int>,<int>,<chr>,<int>,<chr>,<dbl>,<chr>,<chr>,<int>,<date>,<dbl>,<chr>
100000015,2,1,20130425,22,94375,9,496,15.26,1073503884,29,22,2013-04-25,2013,2013-04
100000015,2,2,20130425,22,94726,9,496,13.54,1073503884,29,22,2013-04-25,2013,2013-04
100000015,2,3,20130425,22,94729,9,496,9.95,1073503884,29,22,2013-04-25,2013,2013-04
100000015,3,1,20130528,11,99214,9,41400,114.64,1285600932,11,22,2013-05-28,2013,2013-05
100000015,3,2,20130528,11,93000,9,41400,20.08,1285600932,11,22,2013-05-28,2013,2013-05
100000015,4,1,20130719,22,99213,9,496,51.76,1659344091,29,22,2013-07-19,2013,2013-07


DESY_SORT_KEY,REFERENCE_YEAR,STATE_CODE,COUNTY_CODE,SEX_CODE,RACE_CODE,AGE,ORIG_REASON_FOR_ENTITLEMENT,CURR_REASON_FOR_ENTITLEMENT,ENTITLEMENT_BUY_IN_IND01,ENTITLEMENT_BUY_IN_IND02,ENTITLEMENT_BUY_IN_IND03,ENTITLEMENT_BUY_IN_IND04,ENTITLEMENT_BUY_IN_IND05,ENTITLEMENT_BUY_IN_IND06,ENTITLEMENT_BUY_IN_IND07,ENTITLEMENT_BUY_IN_IND08,ENTITLEMENT_BUY_IN_IND09,ENTITLEMENT_BUY_IN_IND10,ENTITLEMENT_BUY_IN_IND11,ENTITLEMENT_BUY_IN_IND12,HMO_INDICATOR01,HMO_INDICATOR02,HMO_INDICATOR03,HMO_INDICATOR04,HMO_INDICATOR05,HMO_INDICATOR06,HMO_INDICATOR07,HMO_INDICATOR08,HMO_INDICATOR09,HMO_INDICATOR10,HMO_INDICATOR11,HMO_INDICATOR12,VALID_DATE_OF_DEATH_SWITCH,DATE_OF_DEATH,year,date_died,date_died_valid
<int>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>
0,13,45,910,2,1,75,0,0,C,C,C,C,C,C,C,C,C,C,C,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2013,20131017,V
0,13,45,910,2,1,75,0,0,C,C,C,C,C,C,C,C,C,C,C,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2013,20131017,V
0,13,45,910,2,1,75,0,0,C,C,C,C,C,C,C,C,C,C,C,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2013,20150627,V
0,13,45,910,2,1,75,0,0,C,C,C,C,C,C,C,C,C,C,C,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2013,20150627,V
0,13,22,170,2,1,71,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2013,20131017,V
0,13,22,170,2,1,71,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2013,20131017,V


### Loading sample data (for pc)

In [4]:
sample_data <- readRDS(file = "sample_data.RDS")

In [5]:
carrier_data_all_years <- sample_data[[1]]
outpatient_data_all_years <- sample_data[[2]]
inpatient_data_all_years <- sample_data[[3]]
mbsf_data <- read_fst("mbsf_data_long.fst", as.data.table = T)
revenue_center_outpatient_all_years <- sample_data[[5]]
outpatient_and_revenue_center_data <- sample_data[[6]]

### Patient yearly expenditures and use of services carrier

I will first create a function that adds conditions of interest to the data.


#### Finding conditions for each claim line

In [46]:
yearly_calculator_patient_conditions <- function(data) {

  # requirements
  require(data.table)
  require(dtplyr)
  require(tidyverse)
  require(lubridate)

  data %>%
    mutate(
      is_office_visit = HCPCS_CD %in% office_visit_codes,
      is_by_primary_care_physician = PRVDR_SPCLTY %in% primary_care_specialty_codes,
      is_hypertension = if_else(
        LINE_ICD_DGNS_VRSN_CD == 0,
        substr(LINE_ICD_DGNS_CD, 0, 3) %in% hypertension_icd_10_codes,
        if_else(
          LINE_ICD_DGNS_VRSN_CD == 9,
          substr(LINE_ICD_DGNS_CD, 0, 3) %in% hypertension_icd_9_codes, NA
        )
      ),
      is_arthritis = if_else(
        LINE_ICD_DGNS_VRSN_CD == 0,
        substr(LINE_ICD_DGNS_CD, 0, 3) %in% arthritis_icd_10_codes,
        if_else(
          LINE_ICD_DGNS_VRSN_CD == 9,
          substr(LINE_ICD_DGNS_CD, 0, 3) %in% arthritis_icd_9_codes, NA
        )
      ),
      is_IHD = if_else(
        LINE_ICD_DGNS_VRSN_CD == 0,
        substr(LINE_ICD_DGNS_CD, 0, 3) %in% IHD_icd_10_codes,
        if_else(
          LINE_ICD_DGNS_VRSN_CD == 9,
          substr(LINE_ICD_DGNS_CD, 0, 3) %in% IHD_icd_9_codes, NA
        )
      ),
      is_diabetes = if_else(
        LINE_ICD_DGNS_VRSN_CD == 0,
        substr(LINE_ICD_DGNS_CD, 0, 3) %in% diabetes_icd_10_codes,
        if_else(
          LINE_ICD_DGNS_VRSN_CD == 9,
          substr(LINE_ICD_DGNS_CD, 0, 3) %in% diabetes_icd_9_codes, NA
        )
      ),
      is_depression = if_else(
        LINE_ICD_DGNS_VRSN_CD == 0,
        substr(LINE_ICD_DGNS_CD, 0, 3) %in% depression_icd_10_codes,
        if_else(
          LINE_ICD_DGNS_VRSN_CD == 9,
          substr(LINE_ICD_DGNS_CD, 0, 4) %in% depression_icd_9_codes, NA
        )
      )
    ) %>%
    as.data.table()
}

yearly_patient_conditions_carrier <- yearly_calculator_patient_conditions(carrier_data_all_years)
head(yearly_patient_conditions_carrier)

DESY_SORT_KEY,CLAIM_NO,LINE_NUM,CLM_THRU_DT,LINE_PLACE_OF_SRVC_CD,HCPCS_CD,LINE_ICD_DGNS_VRSN_CD,LINE_ICD_DGNS_CD,LINE_ALOWD_CHRG_AMT,PRF_PHYSN_NPI,PRVDR_SPCLTY,PRVDR_STATE_CD,date,year,month_year,is_office_visit,is_by_primary_care_physician,is_hypertension,is_arthritis,is_IHD,is_diabetes,is_depression
<int>,<int>,<int>,<int>,<int>,<chr>,<int>,<chr>,<dbl>,<chr>,<chr>,<int>,<date>,<dbl>,<chr>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
100000015,2,1,20130425,22,94375,9,496,15.26,1073503884,29,22,2013-04-25,2013,2013-04,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
100000015,2,2,20130425,22,94726,9,496,13.54,1073503884,29,22,2013-04-25,2013,2013-04,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
100000015,2,3,20130425,22,94729,9,496,9.95,1073503884,29,22,2013-04-25,2013,2013-04,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
100000015,3,1,20130528,11,99214,9,41400,114.64,1285600932,11,22,2013-05-28,2013,2013-05,TRUE,TRUE,FALSE,FALSE,TRUE,FALSE,FALSE
100000015,3,2,20130528,11,93000,9,41400,20.08,1285600932,11,22,2013-05-28,2013,2013-05,FALSE,TRUE,FALSE,FALSE,TRUE,FALSE,FALSE
100000015,4,1,20130719,22,99213,9,496,51.76,1659344091,29,22,2013-07-19,2013,2013-07,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE


#### Summarizing patient data
I will now summarise the data for each patient.


In [ ]:
summarise_carrier <- function(data, time_frame = 365) {
  data %>%
    group_by(DESY_SORT_KEY, year) %>%
    summarise(
      # tot_allowed_carrier = sum(na.rm = T, LINE_ALOWD_CHRG_AMT),

      # office_visit_count = sum(na.rm = T, is_office_visit),

      # office_visit_cost_carrier = sum(na.rm = T, LINE_ALOWD_CHRG_AMT * is_office_visit),
      distinct_clinicians = length(unique(PRF_PHYSN_NPI)),
      distinct_primary_care_physicians = length(.[is_by_primary_care_physician, unique(PRF_PHYSN_NPI)]),
      hypertension = sum(is_hypertension, na.rm = T) > 0,
      arthritis = sum(is_arthritis, na.rm = T) > 0,
      IHD = sum(is_IHD, na.rm = T) > 0,
      diabetes = sum(is_diabetes, na.rm = T) > 0,
      depression = sum(is_depression, na.rm = T) > 0,
      icd_9_pure = ifelse(prod(LINE_ICD_DGNS_VRSN_CD, na.rm = T) == 0, F, T),
      icd_10_pure = ifelse(sum(LINE_ICD_DGNS_VRSN_CD, na.rm = T) == 0, T, F),
    ) %>%
    as.data.table()
}


summary_patient_by_year <- summarise_carrier(yearly_patient_conditions_carrier)
head(summary_patient_by_year)

`summarise()` has grouped output by 'DESY_SORT_KEY'. You can override using the `.groups` argument.


In [184]:
add_patient_characteristics <- function(mbsf_data, summary_data) {
  mbsf_data <- unique(mbsf_data)
  require(dtplyr)
  require(lubridate)
  require(tidyverse)
  data <- left_join(summary_data, mbsf_data, by = c("DESY_SORT_KEY", "year")) %>% as.data.table()

  data %>%
    mutate(
      year_of_death = substr(date_died, 0, 4)
    ) %>%
    as.data.table()
}

summary_with_patient_characteristics <- add_patient_characteristics(mbsf_data, summary_patient_by_year)
head(summary_with_patient_characteristics)

DESY_SORT_KEY,year,distinct_clinicians,distinct_primary_care_physicians,hypertension,arthritis,IHD,diabetes,depression,icd_9_pure,icd_10_pure,REFERENCE_YEAR,STATE_CODE,COUNTY_CODE,SEX_CODE,RACE_CODE,AGE,ORIG_REASON_FOR_ENTITLEMENT,CURR_REASON_FOR_ENTITLEMENT,ENTITLEMENT_BUY_IN_IND01,ENTITLEMENT_BUY_IN_IND02,ENTITLEMENT_BUY_IN_IND03,ENTITLEMENT_BUY_IN_IND04,ENTITLEMENT_BUY_IN_IND05,ENTITLEMENT_BUY_IN_IND06,ENTITLEMENT_BUY_IN_IND07,ENTITLEMENT_BUY_IN_IND08,ENTITLEMENT_BUY_IN_IND09,ENTITLEMENT_BUY_IN_IND10,ENTITLEMENT_BUY_IN_IND11,ENTITLEMENT_BUY_IN_IND12,HMO_INDICATOR01,HMO_INDICATOR02,HMO_INDICATOR03,HMO_INDICATOR04,HMO_INDICATOR05,HMO_INDICATOR06,HMO_INDICATOR07,HMO_INDICATOR08,HMO_INDICATOR09,HMO_INDICATOR10,HMO_INDICATOR11,HMO_INDICATOR12,VALID_DATE_OF_DEATH_SWITCH,DATE_OF_DEATH,date_died,date_died_valid,year_of_death
<int>,<dbl>,<int>,<int>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>
100000015,2013,7,1,FALSE,FALSE,TRUE,FALSE,FALSE,TRUE,FALSE,13,22,160,1,1,76,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA
100000015,2014,10,1,FALSE,FALSE,TRUE,FALSE,FALSE,TRUE,FALSE,14,22,160,1,1,77,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA
100000015,2015,6,2,TRUE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,15,22,160,1,1,78,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA
100000015,2016,9,2,TRUE,FALSE,TRUE,TRUE,FALSE,FALSE,TRUE,2016,22,160,1,1,79,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA
100000015,2017,14,3,FALSE,FALSE,TRUE,TRUE,FALSE,FALSE,TRUE,2017,22,160,1,1,80,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA
100000015,2018,16,2,FALSE,FALSE,TRUE,TRUE,FALSE,FALSE,TRUE,2018,22,160,1,1,81,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA


In [188]:
write.fst(summary_with_patient_characteristics, "summary_with_patient_characteristics_before_join.fst")
write.fst(summary_patient_by_year, "summary_patient_by_year_before_join.fst")

### Most common physicians for each patient

Now, we will find most common physicians and cardiologists for each patient.


In [ ]:
summary_with_patient_characteristics <- read.fst("summary_with_patient_characteristics_before_join.fst", as.data.table = T)
summary_patient_by_year <- read.fst("summary_patient_by_year_before_join.fst", as.data.table = T)

In [192]:
# adding most common physicians
add_patient_NPI <- function(data, summary_data, time_frame = 365) {
  comorbidity_and_phys_data <-
    inner_join(data, summary_data[, c(
      "DESY_SORT_KEY", "year",
      "icd_9_pure",
      "icd_10_pure"
    )], by = c("DESY_SORT_KEY", "year")) %>%
    as.data.table()

  patient_NPI_count_finder <- function(data) {
    result <- data %>%
      mutate(is_office_visit = HCPCS_CD %in% office_visit_codes) %>%
      group_by(DESY_SORT_KEY, year, PRF_PHYSN_NPI) %>%
      summarise(n = sum(is_office_visit, na.rm = T)) %>%
      filter(n > 0) %>%
      arrange(.by_group = T, desc(n))
  }

  patient_NPI_counts <- patient_NPI_count_finder(comorbidity_and_phys_data)

  patient_NPI_counts <- left_join(patient_NPI_counts,
    distinct(data[, .(PRF_PHYSN_NPI, PRVDR_SPCLTY)]),
    by = "PRF_PHYSN_NPI"
  )

  find_most_common <- function(data) {
    data %>%
      group_by(DESY_SORT_KEY, year) %>%
      arrange(.by_group = T, desc(n)) %>%
      slice(1) %>%
      as.data.table()
  }

  find_most_common_by_specialty <- function(data, specialty_code) {
    data %>%
      filter(PRVDR_SPCLTY %in% specialty_code) %>%
      group_by(DESY_SORT_KEY, year) %>%
      arrange(.by_group = T, desc(n)) %>%
      slice(1) %>%
      as.data.table()
  }

  most_common_physician <- find_most_common(patient_NPI_counts)
  most_common_primary_care_physician <- find_most_common_by_specialty(patient_NPI_counts,
    specialty_code = c("01", "08", "11", "38")
  )
  most_common_physician <- data.frame(most_common_physician) %>%
    rename_with(~ paste0("most_common_physician_", .x))
  most_common_primary_care_physician <- data.frame(most_common_primary_care_physician) %>%
    rename_with(~ paste0("most_common_primary_care_physician_", .x))

  summary_data <- left_join(
    summary_data,
    most_common_physician,
    by = c("DESY_SORT_KEY" = "most_common_physician_DESY_SORT_KEY", "year" = "most_common_physician_year")
  )
  summary_data <- left_join(
    summary_data,
    most_common_primary_care_physician,
    by = c("DESY_SORT_KEY" = "most_common_primary_care_physician_DESY_SORT_KEY", "year" = "most_common_primary_care_physician_year")
  ) %>%
    as.data.table()
}

summary_with_npi <- add_patient_NPI(data = carrier_data_all_years, summary_data = summary_with_patient_characteristics)
head(summary_with_npi)

`summarise()` has grouped output by 'DESY_SORT_KEY', 'year'. You can override using the `.groups` argument.


ERROR: Error: cannot allocate vector of size 5.5 Gb


In [ ]:
write.fst(summary_with_npi, "summary_with_npi_before_join.fst")

## Physician integration status

Here, I will find which physicians are integrated.


### A method of finding codes that are not exclusive to hospitals or non-hospital places
We can exclude these HCPCS codes and only include codes that are not exclusive to hospitals.

In [10]:
exclusive_hospital_code_finder <- function(data,
                                           threshold = 0.05,
                                           integrated_place_of_service_codes = c("19", "22"),
                                           all_place_of_service_codes = c("11", "19", "22")) {
  require(dtplyr)
  require(tidyverse)

  result <- data %>%
    filter(LINE_PLACE_OF_SRVC_CD %in% all_place_of_service_codes) %>%
    group_by(HCPCS_CD) %>%
    summarise(prp_in_facility = nrow(.[LINE_PLACE_OF_SRVC_CD %in% integrated_place_of_service_codes]) / n()) %>%
    as.data.table()

  exclusive_codes <- result[prp_in_facility < threshold | prp_in_facility > (1 - threshold), HCPCS_CD]


  return(exclusive_codes)
}

exclusive_hospital_codes <- exclusive_hospital_code_finder(carrier_data_all_years)

### A function to find integrated docs

In [12]:
# calculate and add physician integration data
# this only uses visits to see if a physician is integrated or not (codde list)

physician_integration_finder <- function(data,
                                         integrated_place_of_service_codes = c("19", "22"),
                                         all_place_of_service_codes = c("11", "19", "22"),
                                         # integration_threshold = 0.5,
                                         office_code_list = c(
                                           "99201",
                                           "99202",
                                           "99203",
                                           "99204",
                                           "99205",
                                           "99211",
                                           "99212",
                                           "99213",
                                           "99214",
                                           "99215"
                                         ),
                                         exclusive_hospital_codes) {
  require(dtplyr)
  require(tidyverse)

  # data = subset(data, HCPCS_CD %in% code_list)
  result <- data %>%
    mutate(
      is_facility = LINE_PLACE_OF_SRVC_CD %in% integrated_place_of_service_codes,
      is_all = LINE_PLACE_OF_SRVC_CD %in% all_place_of_service_codes,
      is_office_visit = HCPCS_CD %in% office_code_list,
      has_non_exclusive_code = HCPCS_CD %!in% exclusive_hospital_codes
    ) %>%
    group_by(PRF_PHYSN_NPI, year) %>%
    summarise(
      in_facility_visits_count = sum(is_facility * is_office_visit, na.rm = T),
      in_all_visits_count = sum(is_all * is_office_visit, na.rm = T),
      in_facility_non_exclusive_HCPCS_count = sum(is_facility * has_non_exclusive_code, na.rm = T),
      in_all_non_exclusive_HCPCS_count = sum(is_all * has_non_exclusive_code, na.rm = T),
      in_facility_count = sum(is_facility, na.rm = T),
      in_all_count = sum(is_all, na.rm = T)
    ) %>%
    mutate(
      in_facility_visits_prp = in_facility_visits_count / in_all_visits_count,
      in_facility_non_exclusive_HCPCS_prp = in_facility_non_exclusive_HCPCS_count / in_all_non_exclusive_HCPCS_count,
      in_facility_prp = in_facility_count / in_all_count
    ) %>%
    as.data.table()
}

physician_integration_stats <- physician_integration_finder(carrier_data_all_years, exclusive_hospital_codes = exclusive_hospital_codes)

`summarise()` has grouped output by 'PRF_PHYSN_NPI'. You can override using the `.groups` argument.


In [13]:
tail(physician_integration_stats)

PRF_PHYSN_NPI,year,in_facility_visits_count,in_all_visits_count,in_facility_non_exclusive_HCPCS_count,in_all_non_exclusive_HCPCS_count,in_facility_count,in_all_count,in_facility_visits_prp,in_facility_non_exclusive_HCPCS_prp,in_facility_prp
<chr>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>
1992997522,2013,2,2,10,10,10,10,1,1,1
1992998157,2013,0,0,0,0,0,0,NaN,NaN,NaN
1992998207,2013,0,0,0,0,0,5,NaN,NaN,0
1992999122,2013,0,8,0,8,0,8,0,0,0
9999999991,2013,0,1,0,2,0,2,0,0,0
9999999992,2013,0,3,0,16,0,71,0,0,0


### Add integration status of physicians
This function will add the integration status of most common physicians to each patient's summary data.

In [148]:
summary_with_npi <- read.fst("./summary_with_npi_before_join.fst", as.data.table = T)

physician_integration_stats <- read.fst("./physician_integration_stats_before_join.fst", as.data.table = T)

In [ ]:
# rename columns
rename_last <- function(data, how_many, new_names) {
  total_cols <- ncol(data)
  setnames(data, (total_cols - how_many + 1):(total_cols), new_names)
}
add_integration_status <- function(data, physician_integration_stats) {
  data_selected <- data[, c(
    "DESY_SORT_KEY",
    "year",
    "most_common_physician_PRF_PHYSN_NPI",
    "most_common_primary_care_physician_PRF_PHYSN_NPI"
  )]

  most_common_physician <- left_join(
    data_selected,
    physician_integration_stats,
    by = c(
      "most_common_physician_PRF_PHYSN_NPI" = "PRF_PHYSN_NPI", "year" = "year"
    )
  ) %>% as.data.table()

  most_common_physician <- most_common_physician[, -c("most_common_primary_care_physician_PRF_PHYSN_NPI")]

  rename_last(
    most_common_physician,
    ncol(physician_integration_stats) - 2,
    paste("most_common_physician_", colnames(physician_integration_stats)[3:ncol(physician_integration_stats)], sep = "")
  )


  most_common_primary_care <- left_join(
    data_selected,
    physician_integration_stats,
    by = c(
      "most_common_primary_care_physician_PRF_PHYSN_NPI" = "PRF_PHYSN_NPI", "year" = "year"
    )
  ) %>% as.data.table()

  most_common_primary_care <- most_common_primary_care[, -c("most_common_physician_PRF_PHYSN_NPI")]

  rename_last(
    most_common_primary_care,
    ncol(physician_integration_stats) - 2,
    paste("most_common_primary_care_physician_", colnames(physician_integration_stats)[3:ncol(physician_integration_stats)], sep = "")
  )

  physician_data <-
    full_join(most_common_physician[, -("most_common_physician_PRF_PHYSN_NPI")],
      most_common_primary_care[, -("most_common_primary_care_physician_PRF_PHYSN_NPI")],
      by = c("DESY_SORT_KEY", "year")
    )
  result <- full_join(data,
    physician_data,
    by = c("DESY_SORT_KEY", "year")
  ) %>%
    as.data.table()


  return(result)
}

summary_with_physician_integration_stats <- add_integration_status(
  data = summary_with_npi,
  physician_integration_stats = physician_integration_stats
)

In [ ]:
head(summary_with_physician_integration_stats)

In [ ]:
write.fst(summary_with_physician_integration_stats, "summary_with_physician_integration_stats.fst")

In [149]:
summary_with_npi[DESY_SORT_KEY == "100005325"]

DESY_SORT_KEY,year,distinct_clinicians,distinct_primary_care_physicians,hypertension,arthritis,IHD,diabetes,depression,icd_9_pure,icd_10_pure,REFERENCE_YEAR,STATE_CODE,COUNTY_CODE,SEX_CODE,RACE_CODE,AGE,ORIG_REASON_FOR_ENTITLEMENT,CURR_REASON_FOR_ENTITLEMENT,ENTITLEMENT_BUY_IN_IND01,ENTITLEMENT_BUY_IN_IND02,ENTITLEMENT_BUY_IN_IND03,ENTITLEMENT_BUY_IN_IND04,ENTITLEMENT_BUY_IN_IND05,ENTITLEMENT_BUY_IN_IND06,ENTITLEMENT_BUY_IN_IND07,ENTITLEMENT_BUY_IN_IND08,ENTITLEMENT_BUY_IN_IND09,ENTITLEMENT_BUY_IN_IND10,ENTITLEMENT_BUY_IN_IND11,ENTITLEMENT_BUY_IN_IND12,HMO_INDICATOR01,HMO_INDICATOR02,HMO_INDICATOR03,HMO_INDICATOR04,HMO_INDICATOR05,HMO_INDICATOR06,HMO_INDICATOR07,HMO_INDICATOR08,HMO_INDICATOR09,HMO_INDICATOR10,HMO_INDICATOR11,HMO_INDICATOR12,VALID_DATE_OF_DEATH_SWITCH,DATE_OF_DEATH,date_died,date_died_valid,year_of_death,most_common_physician_PRF_PHYSN_NPI,most_common_physician_n,most_common_physician_PRVDR_SPCLTY,most_common_primary_care_physician_PRF_PHYSN_NPI,most_common_primary_care_physician_n,most_common_primary_care_physician_PRVDR_SPCLTY
<int>,<dbl>,<int>,<int>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>
100005325,2013,10,3,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,13,20,140,2,1,76,0,0,C,C,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,V,20140125,20140125,V,2014,1578594818,56,11,1578594818,56,11
100005325,2013,10,3,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,13,20,140,2,1,76,0,0,C,C,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,V,20140125,20140125,V,2014,1578594818,56,11,1578594818,56,11
100005325,2013,10,3,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,13,20,140,2,1,76,0,0,C,C,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,V,20140125,20140125,V,2014,1578594818,56,11,1578594818,56,11
100005325,2013,10,3,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,13,20,140,2,1,76,0,0,C,C,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,V,20140125,20140125,V,2014,1578594818,56,11,1578594818,56,11
100005325,2014,13,2,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,14,20,140,2,1,77,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,V,20140125,20140125,V,2014,1578594818,4,11,1578594818,4,11
100005325,2014,13,2,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,14,20,140,2,1,77,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,V,20140125,20140125,V,2014,1578594818,4,11,1578594818,4,11
100005325,2014,13,2,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,14,20,140,2,1,77,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,V,20140125,20140125,V,2014,1578594818,4,11,1578594818,4,11
100005325,2014,13,2,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,14,20,140,2,1,77,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,V,20140125,20140125,V,2014,1578594818,4,11,1578594818,4,11


# Comparisons and analyses

## Reading the patient and physician integration results

In [3]:
yearly_calculations <-
  read_fst("calculation_results.fst", as.data.table = T)

yearly_calculations <- unique(yearly_calculations)

In [10]:
yearly_calculations[DESY_SORT_KEY == "100005325"]

DESY_SORT_KEY,year,distinct_clinicians,distinct_primary_care_physicians,hypertension,arthritis,IHD,diabetes,depression,icd_9_pure,icd_10_pure,REFERENCE_YEAR,STATE_CODE,COUNTY_CODE,SEX_CODE,RACE_CODE,AGE,ORIG_REASON_FOR_ENTITLEMENT,CURR_REASON_FOR_ENTITLEMENT,ENTITLEMENT_BUY_IN_IND01,ENTITLEMENT_BUY_IN_IND02,ENTITLEMENT_BUY_IN_IND03,ENTITLEMENT_BUY_IN_IND04,ENTITLEMENT_BUY_IN_IND05,ENTITLEMENT_BUY_IN_IND06,ENTITLEMENT_BUY_IN_IND07,ENTITLEMENT_BUY_IN_IND08,ENTITLEMENT_BUY_IN_IND09,ENTITLEMENT_BUY_IN_IND10,ENTITLEMENT_BUY_IN_IND11,ENTITLEMENT_BUY_IN_IND12,HMO_INDICATOR01,HMO_INDICATOR02,HMO_INDICATOR03,HMO_INDICATOR04,HMO_INDICATOR05,HMO_INDICATOR06,HMO_INDICATOR07,HMO_INDICATOR08,HMO_INDICATOR09,HMO_INDICATOR10,HMO_INDICATOR11,HMO_INDICATOR12,VALID_DATE_OF_DEATH_SWITCH,DATE_OF_DEATH,date_died,date_died_valid,year_of_death,most_common_physician_PRF_PHYSN_NPI,most_common_physician_n,most_common_physician_PRVDR_SPCLTY,most_common_primary_care_physician_PRF_PHYSN_NPI,most_common_primary_care_physician_n,most_common_primary_care_physician_PRVDR_SPCLTY,most_common_physician_in_facility_visits_count,most_common_physician_in_all_visits_count,most_common_physician_in_facility_non_exclusive_HCPCS_count,most_common_physician_in_all_non_exclusive_HCPCS_count,most_common_physician_in_facility_count,most_common_physician_in_all_count,most_common_physician_in_facility_visits_prp,most_common_physician_in_facility_non_exclusive_HCPCS_prp,most_common_physician_in_facility_prp,most_common_primary_care_physician_in_facility_visits_count,most_common_primary_care_physician_in_all_visits_count,most_common_primary_care_physician_in_facility_non_exclusive_HCPCS_count,most_common_primary_care_physician_in_all_non_exclusive_HCPCS_count,most_common_primary_care_physician_in_facility_count,most_common_primary_care_physician_in_all_count,most_common_primary_care_physician_in_facility_visits_prp,most_common_primary_care_physician_in_facility_non_exclusive_HCPCS_prp,most_common_primary_care_physician_in_facility_prp
<int>,<dbl>,<int>,<int>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>
100005325,2013,10,3,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,13,20,140,2,1,76,0,0,C,C,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,V,20140125,20140125,V,2014,1578594818,14,11,1578594818,14,11,1,163,36,199,46,384,0.006134969,0.1809045,0.11979167,1,163,36,199,46,384,0.006134969,0.1809045,0.11979167
100005325,2014,13,2,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,14,20,140,2,1,77,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,V,20140125,20140125,V,2014,1578594818,1,11,1578594818,1,11,0,132,26,163,26,290,0.000000000,0.1595092,0.08965517,0,132,26,163,26,290,0.000000000,0.1595092,0.08965517


## Adding metropolitan status
I will add the metropolitan statuses of the patient counties using the USDA data

In [11]:
rural_urban_data <- readxl::read_xls("physician_data/ruralurbancodes2013.xls") %>% as.data.table()
head(rural_urban_data)
cross_walk_rural_urban <- read.csv(file = "physician_data/xwalk2018.csv") %>% as.data.table()
head(cross_walk_rural_urban)

FIPS,State,County_Name,Population_2010,RUCC_2013,Description
<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>
01001,AL,Autauga County,54571,2,"Metro - Counties in metro areas of 250,000 to 1 million population"
01003,AL,Baldwin County,182265,3,"Metro - Counties in metro areas of fewer than 250,000 population"
01005,AL,Barbour County,27457,6,"Nonmetro - Urban population of 2,500 to 19,999, adjacent to a metro area"
01007,AL,Bibb County,22915,1,Metro - Counties in metro areas of 1 million population or more
01009,AL,Blount County,57322,1,Metro - Counties in metro areas of 1 million population or more
01011,AL,Bullock County,10914,6,"Nonmetro - Urban population of 2,500 to 19,999, adjacent to a metro area"


County.Name,State,SSACD,FIPS.County.Code,CBSA,CBSA.Name
<chr>,<chr>,<int>,<int>,<int>,<chr>
AUTAUGA,AL,1000,1001,33860,"Montgomery, AL"
BALDWIN,AL,1010,1003,19300,"Daphne-Fairhope-Foley, AL"
BARBOUR,AL,1020,1005,NA,
BIBB,AL,1030,1007,13820,"Birmingham-Hoover, AL"
BLOUNT,AL,1040,1009,13820,"Birmingham-Hoover, AL"
BULLOCK,AL,1050,1011,NA,


In [12]:
yearly_calculations[, SSACD := as.integer(paste(STATE_CODE, COUNTY_CODE, sep = ""))]
yearly_calculations <- left_join(yearly_calculations, cross_walk_rural_urban, by = "SSACD") %>%
  mutate(FIPS.County.Code = as.character(FIPS.County.Code)) %>%
  left_join(., rural_urban_data, by = c("FIPS.County.Code" = "FIPS")) %>%
  as.data.table()
yearly_calculations[, c("SSACD", "County.Name", "State.x", "FIPS.County.Code", "CBSA", "CBSA.Name", "State.y", "County_Name", "Population_2010", "Description") := NULL]
head(yearly_calculations)


# STATE_CODE	COUNTY_CODE	SEX_CODE	RACE_CODE

Warning message in eval(jsub, SDenv, parent.frame()):
“NAs introduced by coercion”


DESY_SORT_KEY,year,distinct_clinicians,distinct_primary_care_physicians,hypertension,arthritis,IHD,diabetes,depression,icd_9_pure,icd_10_pure,REFERENCE_YEAR,STATE_CODE,COUNTY_CODE,SEX_CODE,RACE_CODE,AGE,ORIG_REASON_FOR_ENTITLEMENT,CURR_REASON_FOR_ENTITLEMENT,ENTITLEMENT_BUY_IN_IND01,ENTITLEMENT_BUY_IN_IND02,ENTITLEMENT_BUY_IN_IND03,ENTITLEMENT_BUY_IN_IND04,ENTITLEMENT_BUY_IN_IND05,ENTITLEMENT_BUY_IN_IND06,ENTITLEMENT_BUY_IN_IND07,ENTITLEMENT_BUY_IN_IND08,ENTITLEMENT_BUY_IN_IND09,ENTITLEMENT_BUY_IN_IND10,ENTITLEMENT_BUY_IN_IND11,ENTITLEMENT_BUY_IN_IND12,HMO_INDICATOR01,HMO_INDICATOR02,HMO_INDICATOR03,HMO_INDICATOR04,HMO_INDICATOR05,HMO_INDICATOR06,HMO_INDICATOR07,HMO_INDICATOR08,HMO_INDICATOR09,HMO_INDICATOR10,HMO_INDICATOR11,HMO_INDICATOR12,VALID_DATE_OF_DEATH_SWITCH,DATE_OF_DEATH,date_died,date_died_valid,year_of_death,most_common_physician_PRF_PHYSN_NPI,most_common_physician_n,most_common_physician_PRVDR_SPCLTY,most_common_primary_care_physician_PRF_PHYSN_NPI,most_common_primary_care_physician_n,most_common_primary_care_physician_PRVDR_SPCLTY,most_common_physician_in_facility_visits_count,most_common_physician_in_all_visits_count,most_common_physician_in_facility_non_exclusive_HCPCS_count,most_common_physician_in_all_non_exclusive_HCPCS_count,most_common_physician_in_facility_count,most_common_physician_in_all_count,most_common_physician_in_facility_visits_prp,most_common_physician_in_facility_non_exclusive_HCPCS_prp,most_common_physician_in_facility_prp,most_common_primary_care_physician_in_facility_visits_count,most_common_primary_care_physician_in_all_visits_count,most_common_primary_care_physician_in_facility_non_exclusive_HCPCS_count,most_common_primary_care_physician_in_all_non_exclusive_HCPCS_count,most_common_primary_care_physician_in_facility_count,most_common_primary_care_physician_in_all_count,most_common_primary_care_physician_in_facility_visits_prp,most_common_primary_care_physician_in_facility_non_exclusive_HCPCS_prp,most_common_primary_care_physician_in_facility_prp,RUCC_2013
<int>,<dbl>,<int>,<int>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
100000015,2013,7,1,FALSE,FALSE,TRUE,FALSE,FALSE,TRUE,FALSE,13,22,160,1,1,76,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA,1285600932,2,11,1285600932,2,11,7,96,62,242,66,341,0.07291667,0.2561983,0.19354839,7,96,62,242,66,341,0.07291667,0.2561983,0.19354839,1
100000015,2014,10,1,FALSE,FALSE,TRUE,FALSE,FALSE,TRUE,FALSE,14,22,160,1,1,77,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA,1770514119,4,11,1770514119,4,11,16,16,22,22,27,27,1.00000000,1.0000000,1.00000000,16,16,22,22,27,27,1.00000000,1.0000000,1.00000000,1
100000015,2015,6,2,TRUE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,15,22,160,1,1,78,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA,1770514119,4,11,1770514119,4,11,48,48,57,57,57,57,1.00000000,1.0000000,1.00000000,48,48,57,57,57,57,1.00000000,1.0000000,1.00000000,1
100000015,2016,9,2,TRUE,FALSE,TRUE,TRUE,FALSE,FALSE,TRUE,2016,22,160,1,1,79,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA,1730170630,4,06,1730170630,4,11,0,21,0,21,4,41,0.00000000,0.0000000,0.09756098,0,21,0,21,4,41,0.00000000,0.0000000,0.09756098,1
100000015,2017,14,3,FALSE,FALSE,TRUE,TRUE,FALSE,FALSE,TRUE,2017,22,160,1,1,80,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA,1043207350,3,11,1043207350,3,11,58,58,68,68,69,69,1.00000000,1.0000000,1.00000000,58,58,68,68,69,69,1.00000000,1.0000000,1.00000000,1
100000015,2018,16,2,FALSE,FALSE,TRUE,TRUE,FALSE,FALSE,TRUE,2018,22,160,1,1,81,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA,10432

## Adding sex, race, and state names from codes

In [13]:
add_personal_details <- function(data) {
  require(tidyverse)
  require(dtplyr)
  require(lubridate)

  result <- data %>%
    # left_join(.,census_and_state_codes[,-1],by=c("STATE_CODE"="state_code"))%>%
    left_join(., race_codes, by = c("RACE_CODE" = "race_code")) %>%
    left_join(., sex_codes, by = c("SEX_CODE" = "sex_code")) %>%
    mutate(
      age_group = case_when(
        AGE < 75 & AGE > 64 ~ "65-74",
        AGE > 74 & AGE < 85 ~ "75-84",
        AGE > 84 ~ "85+"
      ),
      urban = (RUCC_2013 <= 3)
    ) %>%
    as.data.table()

  return(result)
}

# STATE_CODE	COUNTY_CODE	SEX_CODE	RACE_CODE

In [14]:
yearly_calculations <- add_personal_details(yearly_calculations)

Loading required package: lubridate


Attaching package: ‘lubridate’


The following object is masked from ‘package:reshape’:

    stamp


The following objects are masked from ‘package:data.table’:

    hour, isoweek, mday, minute, month, quarter, second, wday, week, yday, year


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




## Adding HMO indicator and buy in sums

In [15]:
yearly_calculations[, HMO_INDICATOR_sum := sum(
  HMO_INDICATOR01 == 0,
  HMO_INDICATOR02 == 0,
  HMO_INDICATOR03 == 0,
  HMO_INDICATOR04 == 0,
  HMO_INDICATOR05 == 0,
  HMO_INDICATOR06 == 0,
  HMO_INDICATOR07 == 0,
  HMO_INDICATOR08 == 0,
  HMO_INDICATOR09 == 0,
  HMO_INDICATOR10 == 0,
  HMO_INDICATOR11 == 0,
  HMO_INDICATOR12 == 0,
  na.rm = T
), by = 1:nrow(yearly_calculations)]

yearly_calculations[, ENTITLEMENT_BUY_IN_IND_sum := sum(
  ENTITLEMENT_BUY_IN_IND01 %in% c("3", "C"),
  ENTITLEMENT_BUY_IN_IND02 %in% c("3", "C"),
  ENTITLEMENT_BUY_IN_IND03 %in% c("3", "C"),
  ENTITLEMENT_BUY_IN_IND04 %in% c("3", "C"),
  ENTITLEMENT_BUY_IN_IND05 %in% c("3", "C"),
  ENTITLEMENT_BUY_IN_IND06 %in% c("3", "C"),
  ENTITLEMENT_BUY_IN_IND07 %in% c("3", "C"),
  ENTITLEMENT_BUY_IN_IND08 %in% c("3", "C"),
  ENTITLEMENT_BUY_IN_IND09 %in% c("3", "C"),
  ENTITLEMENT_BUY_IN_IND10 %in% c("3", "C"),
  ENTITLEMENT_BUY_IN_IND11 %in% c("3", "C"),
  ENTITLEMENT_BUY_IN_IND12 %in% c("3", "C"),
  na.rm = T
), by = 1:nrow(yearly_calculations)]

## Adding months alive within the year of death

In [16]:
yearly_calculations[, months_alive := as.numeric(substr(DATE_OF_DEATH, 5, 6))]

## Filtering the data for analysis

In [17]:
length(yearly_calculations$DESY_SORT_KEY)

[1] 14342911

In [18]:
data_for_modelling_filter <- function(data) {
  library(tidyverse)
  library(dtplyr)

  data %>%
    filter(STATE_CODE %!in% non_us_state_codes &

      AGE >= 65 &

      ((!is.na(months_alive) & HMO_INDICATOR_sum >= months_alive) |
        (is.na(months_alive) & HMO_INDICATOR_sum == 12)) &

      ((!is.na(months_alive) & ENTITLEMENT_BUY_IN_IND_sum >= months_alive) |
        (is.na(months_alive) & ENTITLEMENT_BUY_IN_IND_sum == 12))) %>%
    as.data.table()
}

In [19]:
yearly_calculations <- data_for_modelling_filter(yearly_calculations)

In [20]:
length(yearly_calculations$DESY_SORT_KEY)

[1] 10012783

## Counting number of conditions

In [21]:
yearly_calculations[, number_of_conditions := sum(hypertension, arthritis, IHD, diabetes, depression, na.rm = T), ,
  by = 1:nrow(yearly_calculations)
]

In [22]:
head(yearly_calculations, 20)

DESY_SORT_KEY,year,distinct_clinicians,distinct_primary_care_physicians,hypertension,arthritis,IHD,diabetes,depression,icd_9_pure,icd_10_pure,REFERENCE_YEAR,STATE_CODE,COUNTY_CODE,SEX_CODE,RACE_CODE,AGE,ORIG_REASON_FOR_ENTITLEMENT,CURR_REASON_FOR_ENTITLEMENT,ENTITLEMENT_BUY_IN_IND01,ENTITLEMENT_BUY_IN_IND02,ENTITLEMENT_BUY_IN_IND03,ENTITLEMENT_BUY_IN_IND04,ENTITLEMENT_BUY_IN_IND05,ENTITLEMENT_BUY_IN_IND06,ENTITLEMENT_BUY_IN_IND07,ENTITLEMENT_BUY_IN_IND08,ENTITLEMENT_BUY_IN_IND09,ENTITLEMENT_BUY_IN_IND10,ENTITLEMENT_BUY_IN_IND11,ENTITLEMENT_BUY_IN_IND12,HMO_INDICATOR01,HMO_INDICATOR02,HMO_INDICATOR03,HMO_INDICATOR04,HMO_INDICATOR05,HMO_INDICATOR06,HMO_INDICATOR07,HMO_INDICATOR08,HMO_INDICATOR09,HMO_INDICATOR10,HMO_INDICATOR11,HMO_INDICATOR12,VALID_DATE_OF_DEATH_SWITCH,DATE_OF_DEATH,date_died,date_died_valid,year_of_death,most_common_physician_PRF_PHYSN_NPI,most_common_physician_n,most_common_physician_PRVDR_SPCLTY,most_common_primary_care_physician_PRF_PHYSN_NPI,most_common_primary_care_physician_n,most_common_primary_care_physician_PRVDR_SPCLTY,most_common_physician_in_facility_visits_count,most_common_physician_in_all_visits_count,most_common_physician_in_facility_non_exclusive_HCPCS_count,most_common_physician_in_all_non_exclusive_HCPCS_count,most_common_physician_in_facility_count,most_common_physician_in_all_count,most_common_physician_in_facility_visits_prp,most_common_physician_in_facility_non_exclusive_HCPCS_prp,most_common_physician_in_facility_prp,most_common_primary_care_physician_in_facility_visits_count,most_common_primary_care_physician_in_all_visits_count,most_common_primary_care_physician_in_facility_non_exclusive_HCPCS_count,most_common_primary_care_physician_in_all_non_exclusive_HCPCS_count,most_common_primary_care_physician_in_facility_count,most_common_primary_care_physician_in_all_count,most_common_primary_care_physician_in_facility_visits_prp,most_common_primary_care_physician_in_facility_non_exclusive_HCPCS_prp,most_common_primary_care_physician_in_facility_prp,RUCC_2013,race,sex,age_group,urban,HMO_INDICATOR_sum,ENTITLEMENT_BUY_IN_IND_sum,months_alive,number_of_conditions
<int>,<dbl>,<int>,<int>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<dbl>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<lgl>,<int>,<int>,<dbl>,<int>
100000015,2013,7,1,FALSE,FALSE,TRUE,FALSE,FALSE,TRUE,FALSE,13,22,160,1,1,76,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA,1285600932,2,11,1285600932,2,11,7,96,62,242,66,341,0.07291667,0.25619835,0.19354839,7,96,62,242,66,341,0.07291667,0.25619835,0.19354839,1,White,Male,75-84,TRUE,12,12,NA,1
100000015,2014,10,1,FALSE,FALSE,TRUE,FALSE,FALSE,TRUE,FALSE,14,22,160,1,1,77,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA,1770514119,4,11,1770514119,4,11,16,16,22,22,27,27,1.00000000,1.00000000,1.00000000,16,16,22,22,27,27,1.00000000,1.00000000,1.00000000,1,White,Male,75-84,TRUE,12,12,NA,1
100000015,2015,6,2,TRUE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,15,22,160,1,1,78,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA,1770514119,4,11,1770514119,4,11,48,48,57,57,57,57,1.00000000,1.00000000,1.00000000,48,48,57,57,57,57,1.00000000,1.00000000,1.00000000,1,White,Male,75-84,TRUE,12,12,NA,2
100000015,2016,9,2,TRUE,FALSE,TRUE,TRUE,FALSE,FALSE,TRUE,2016,22,160,1,1,79,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA,1730170630,4,06,1730170630,4,11,0,21,0,21,4,41,0.00000000,0.00000000,0.09756098,0,21,0,21,4,41,0.00000000,0.00000000,0.09756098,1,White,Male,75-84,TRUE,12,12,NA,3
100000015,2017,14,3,FALSE,FALSE,TRUE,TRUE,FALSE,FALSE,TRUE,2017,22,160,1,1,80,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,N

In [23]:
write_fst(yearly_calculations, "yearly_calculations_cleaned.fst")

# Read cleaned results

In [143]:
yearly_calculations <- read_fst("yearly_calculations_cleaned.fst", as.data.table = T)

In [144]:
yearly_calculations[DESY_SORT_KEY == "100005325"]

DESY_SORT_KEY,year,distinct_clinicians,distinct_primary_care_physicians,hypertension,arthritis,IHD,diabetes,depression,icd_9_pure,icd_10_pure,REFERENCE_YEAR,STATE_CODE,COUNTY_CODE,SEX_CODE,RACE_CODE,AGE,ORIG_REASON_FOR_ENTITLEMENT,CURR_REASON_FOR_ENTITLEMENT,ENTITLEMENT_BUY_IN_IND01,ENTITLEMENT_BUY_IN_IND02,ENTITLEMENT_BUY_IN_IND03,ENTITLEMENT_BUY_IN_IND04,ENTITLEMENT_BUY_IN_IND05,ENTITLEMENT_BUY_IN_IND06,ENTITLEMENT_BUY_IN_IND07,ENTITLEMENT_BUY_IN_IND08,ENTITLEMENT_BUY_IN_IND09,ENTITLEMENT_BUY_IN_IND10,ENTITLEMENT_BUY_IN_IND11,ENTITLEMENT_BUY_IN_IND12,HMO_INDICATOR01,HMO_INDICATOR02,HMO_INDICATOR03,HMO_INDICATOR04,HMO_INDICATOR05,HMO_INDICATOR06,HMO_INDICATOR07,HMO_INDICATOR08,HMO_INDICATOR09,HMO_INDICATOR10,HMO_INDICATOR11,HMO_INDICATOR12,VALID_DATE_OF_DEATH_SWITCH,DATE_OF_DEATH,date_died,date_died_valid,year_of_death,most_common_physician_PRF_PHYSN_NPI,most_common_physician_n,most_common_physician_PRVDR_SPCLTY,most_common_primary_care_physician_PRF_PHYSN_NPI,most_common_primary_care_physician_n,most_common_primary_care_physician_PRVDR_SPCLTY,most_common_physician_in_facility_visits_count,most_common_physician_in_all_visits_count,most_common_physician_in_facility_non_exclusive_HCPCS_count,most_common_physician_in_all_non_exclusive_HCPCS_count,most_common_physician_in_facility_count,most_common_physician_in_all_count,most_common_physician_in_facility_visits_prp,most_common_physician_in_facility_non_exclusive_HCPCS_prp,most_common_physician_in_facility_prp,most_common_primary_care_physician_in_facility_visits_count,most_common_primary_care_physician_in_all_visits_count,most_common_primary_care_physician_in_facility_non_exclusive_HCPCS_count,most_common_primary_care_physician_in_all_non_exclusive_HCPCS_count,most_common_primary_care_physician_in_facility_count,most_common_primary_care_physician_in_all_count,most_common_primary_care_physician_in_facility_visits_prp,most_common_primary_care_physician_in_facility_non_exclusive_HCPCS_prp,most_common_primary_care_physician_in_facility_prp,RUCC_2013,race,sex,age_group,urban,HMO_INDICATOR_sum,ENTITLEMENT_BUY_IN_IND_sum,months_alive,number_of_conditions
<int>,<dbl>,<int>,<int>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<dbl>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<lgl>,<int>,<int>,<dbl>,<int>
100005325,2013,10,3,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,13,20,140,2,1,76,0,0,C,C,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,V,20140125,20140125,V,2014,1578594818,56,11,1578594818,56,11,1,163,36,199,46,384,0.006134969,0.1809045,0.1197917,1,163,36,199,46,384,0.006134969,0.1809045,0.1197917,7,White,Female,75-84,FALSE,12,12,1,0
100005325,2013,10,3,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,13,20,140,2,1,76,0,0,C,C,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,V,20140125,20140125,V,2014,1578594818,56,11,1578594818,56,11,1,163,36,199,46,384,0.006134969,0.1809045,0.1197917,1,163,36,199,46,384,0.006134969,0.1809045,0.1197917,7,White,Female,75-84,FALSE,12,12,1,0
100005325,2013,10,3,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,13,20,140,2,1,76,0,0,C,C,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,V,20140125,20140125,V,2014,1578594818,56,11,1578594818,56,11,1,163,36,199,46,384,0.006134969,0.1809045,0.1197917,1,163,36,199,46,384,0.006134969,0.1809045,0.1197917,7,White,Female,75-84,FALSE,12,12,1,0
100005325,2013,10,3,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,13,20,140,2,1,76,0,0,C,C,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,V,20140125,20140125,V,2014,1578594818,56,11,1578594818,56,11,1,163,36,199,46,384,0.006134969,0.1809045,0.1197917,1,163,36,199,46,384,0.006134969,0.1809045,0.1197917,7,White,Female,75-84,FALSE,12,12,1,0
100005325,2013,10,3,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE

In [4]:
head(yearly_calculations)

DESY_SORT_KEY,year,distinct_clinicians,distinct_primary_care_physicians,hypertension,arthritis,IHD,diabetes,depression,icd_9_pure,icd_10_pure,REFERENCE_YEAR,STATE_CODE,COUNTY_CODE,SEX_CODE,RACE_CODE,AGE,ORIG_REASON_FOR_ENTITLEMENT,CURR_REASON_FOR_ENTITLEMENT,ENTITLEMENT_BUY_IN_IND01,ENTITLEMENT_BUY_IN_IND02,ENTITLEMENT_BUY_IN_IND03,ENTITLEMENT_BUY_IN_IND04,ENTITLEMENT_BUY_IN_IND05,ENTITLEMENT_BUY_IN_IND06,ENTITLEMENT_BUY_IN_IND07,ENTITLEMENT_BUY_IN_IND08,ENTITLEMENT_BUY_IN_IND09,ENTITLEMENT_BUY_IN_IND10,ENTITLEMENT_BUY_IN_IND11,ENTITLEMENT_BUY_IN_IND12,HMO_INDICATOR01,HMO_INDICATOR02,HMO_INDICATOR03,HMO_INDICATOR04,HMO_INDICATOR05,HMO_INDICATOR06,HMO_INDICATOR07,HMO_INDICATOR08,HMO_INDICATOR09,HMO_INDICATOR10,HMO_INDICATOR11,HMO_INDICATOR12,VALID_DATE_OF_DEATH_SWITCH,DATE_OF_DEATH,date_died,date_died_valid,year_of_death,most_common_physician_PRF_PHYSN_NPI,most_common_physician_n,most_common_physician_PRVDR_SPCLTY,most_common_primary_care_physician_PRF_PHYSN_NPI,most_common_primary_care_physician_n,most_common_primary_care_physician_PRVDR_SPCLTY,most_common_physician_in_facility_visits_count,most_common_physician_in_all_visits_count,most_common_physician_in_facility_non_exclusive_HCPCS_count,most_common_physician_in_all_non_exclusive_HCPCS_count,most_common_physician_in_facility_count,most_common_physician_in_all_count,most_common_physician_in_facility_visits_prp,most_common_physician_in_facility_non_exclusive_HCPCS_prp,most_common_physician_in_facility_prp,most_common_primary_care_physician_in_facility_visits_count,most_common_primary_care_physician_in_all_visits_count,most_common_primary_care_physician_in_facility_non_exclusive_HCPCS_count,most_common_primary_care_physician_in_all_non_exclusive_HCPCS_count,most_common_primary_care_physician_in_facility_count,most_common_primary_care_physician_in_all_count,most_common_primary_care_physician_in_facility_visits_prp,most_common_primary_care_physician_in_facility_non_exclusive_HCPCS_prp,most_common_primary_care_physician_in_facility_prp,RUCC_2013,race,sex,age_group,urban,HMO_INDICATOR_sum,ENTITLEMENT_BUY_IN_IND_sum,months_alive,number_of_conditions
<int>,<dbl>,<int>,<int>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<dbl>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<lgl>,<int>,<int>,<dbl>,<int>
100000015,2013,7,1,FALSE,FALSE,TRUE,FALSE,FALSE,TRUE,FALSE,13,22,160,1,1,76,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA,1285600932,2,11,1285600932,2,11,7,96,62,242,66,341,0.07291667,0.2561983,0.19354839,7,96,62,242,66,341,0.07291667,0.2561983,0.19354839,1,White,Male,75-84,TRUE,12,12,NA,1
100000015,2014,10,1,FALSE,FALSE,TRUE,FALSE,FALSE,TRUE,FALSE,14,22,160,1,1,77,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA,1770514119,4,11,1770514119,4,11,16,16,22,22,27,27,1.00000000,1.0000000,1.00000000,16,16,22,22,27,27,1.00000000,1.0000000,1.00000000,1,White,Male,75-84,TRUE,12,12,NA,1
100000015,2015,6,2,TRUE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,15,22,160,1,1,78,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA,1770514119,4,11,1770514119,4,11,48,48,57,57,57,57,1.00000000,1.0000000,1.00000000,48,48,57,57,57,57,1.00000000,1.0000000,1.00000000,1,White,Male,75-84,TRUE,12,12,NA,2
100000015,2016,9,2,TRUE,FALSE,TRUE,TRUE,FALSE,FALSE,TRUE,2016,22,160,1,1,79,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA,1730170630,4,06,1730170630,4,11,0,21,0,21,4,41,0.00000000,0.0000000,0.09756098,0,21,0,21,4,41,0.00000000,0.0000000,0.09756098,1,White,Male,75-84,TRUE,12,12,NA,3
100000015,2017,14,3,FALSE,FALSE,TRUE,TRUE,FALSE,FALSE,TRUE,2017,22,160,1,1,80,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA,

# Comparison functions

In [78]:
comparator_averages <- function(data, specialty, integration_threshold = 0.75, independece_threshold = 0.75) {
  require(tidyverse)
  require(dtplyr)
  varname <- paste(specialty, "_in_facility_non_exclusive_HCPCS_prp", sep = "")
  looking_for <- paste(specialty, "_PRF_PHYSN_NPI", sep = "")


  result <- data %>%
    mutate(
      integration_status =
        case_when(
          !!rlang::sym(varname) < independece_threshold ~ paste("Independent ", specialty, sep = ""),
          !!rlang::sym(varname) >= integration_threshold ~ paste("Integrated ", specialty, sep = ""),
          !!rlang::sym(varname) > independece_threshold &
            !!rlang::sym(varname) < integration_threshold ~ paste("Switching ", specialty, sep = ""),
          TRUE ~ paste("Missing ", specialty, sep = "")
        ),
    ) %>%
    group_by(integration_status) %>%
    summarise(
      n = n(),
      N_distinct_patients = length(unique(DESY_SORT_KEY)),
      N_distinct_specialist = length(unique(!!rlang::sym(looking_for))),
      Male = mean(sex == "Male", na.rm = T),
      Female = mean(sex == "Female", na.rm = T),
      Asian = mean(race == "Asian", na.rm = T),
      Black = mean(race == "Black", na.rm = T),
      Hispanic = mean(race == "Hispanic", na.rm = T),
      North_American_Native = mean(race == "North American Native", na.rm = T),
      White = mean(race == "White", na.rm = T),
      Other_race = mean(race == "Other", na.rm = T),
      avg_age = mean(AGE, na.rm = T),
      age_65_74 = mean(na.rm = T, age_group == "65-74"),
      age_75_84 = mean(na.rm = T, age_group == "75-84"),
      age_85_plus = mean(na.rm = T, age_group == "85+"),

      # West=mean(na.rm=T,census_region=="West"),
      # South=mean(na.rm=T,census_region=="South"),
      # Northeast=mean(na.rm=T,census_region=="Northeast"),
      # Midwest=mean(na.rm=T,census_region=="Midwest"),

      rural_prp = mean(RUCC_2013 > 3, na.rm = T),
      urban_prp = mean(RUCC_2013 <= 3, na.rm = T),
      hypertension = mean(hypertension, na.rm = T),
      arthritis = mean(arthritis, na.rm = T),
      IHD = mean(IHD, na.rm = T),
      diabetes = mean(diabetes, na.rm = T),
      depression = mean(depression, na.rm = T),
      two_conditions = mean(number_of_conditions == 2, na.rm = T),
      more_than_two_conditions = mean(number_of_conditions > 2, na.rm = T),
      primary_care_physician_integrated =
        sum(most_common_primary_care_physician_in_facility_non_exclusive_HCPCS_prp >= integration_threshold, na.rm = T) / n(),
      most_common_physician_integrated =
        sum(most_common_physician_in_facility_non_exclusive_HCPCS_prp >= integration_threshold, na.rm = T) / n()
    ) %>%
    as.data.table()

  return(result)
}

In [91]:
comparator_tests <- function(data, specialty, integration_threshold = 0.75, independece_threshold = 0.75) {
  require(tidyverse)
  require(dtplyr)
  varname <- paste(specialty, "_in_facility_non_exclusive_HCPCS_prp", sep = "")
  looking_for <- paste(specialty, "_PRF_PHYSN_NPI", sep = "")


  result <- data %>%
    mutate(
      integration_status =
        case_when(
          !!rlang::sym(varname) < independece_threshold ~ paste("Independent ", specialty, sep = ""),
          !!rlang::sym(varname) >= integration_threshold ~ paste("Integrated ", specialty, sep = "")
        ),
    ) %>%
    summarise(
      Male = chisq.test(sex == "Male", integration_status)$p.value,
      Female = chisq.test(sex == "Female", integration_status)$p.value,
      Asian = chisq.test(race == "Asian", integration_status)$p.value,
      Black = chisq.test(race == "Black", integration_status)$p.value,
      Hispanic = chisq.test(race == "Hispanic", integration_status)$p.value,
      North_American_Native = chisq.test(race == "North American Native", integration_status)$p.value,
      White = chisq.test(race == "White", integration_status)$p.value,
      Other_race = chisq.test(race == "Other", integration_status)$p.value,
      avg_age = t.test(AGE ~ integration_status, data = .)$p.value,
      age_65_74 = chisq.test(age_group == "65-74", integration_status)$p.value,
      age_75_84 = chisq.test(age_group == "75-84", integration_status)$p.value,
      age_85_plus = chisq.test(age_group == "85+", integration_status)$p.value,

      # West=mean(na.rm=T,census_region=="West"),
      # South=mean(na.rm=T,census_region=="South"),
      # Northeast=mean(na.rm=T,census_region=="Northeast"),
      # Midwest=mean(na.rm=T,census_region=="Midwest"),

      rural_prp = chisq.test(RUCC_2013 > 3, integration_status)$p.value,
      urban_prp = chisq.test(RUCC_2013 <= 3, integration_status)$p.value,
      hypertension = chisq.test(hypertension, integration_status)$p.value,
      arthritis = chisq.test(arthritis, integration_status)$p.value,
      IHD = chisq.test(IHD, integration_status)$p.value,
      diabetes = chisq.test(diabetes, integration_status)$p.value,
      depression = chisq.test(depression, integration_status)$p.value,
      two_conditions = chisq.test(number_of_conditions == 2, integration_status)$p.value,
      more_than_two_conditions = chisq.test(number_of_conditions > 2, integration_status)$p.value,
    ) %>%
    as.data.table()

  return(result)
}

In [92]:
primary_care_physician_comparisons_test <-
  comparator_tests(yearly_calculations, "most_common_primary_care_physician", integration_threshold = 0.75, independece_threshold = 0.75)

In [93]:
primary_care_physician_comparisons_test

Male,Female,Asian,Black,Hispanic,North_American_Native,White,Other_race,avg_age,age_65_74,age_75_84,age_85_plus,rural_prp,urban_prp,hypertension,arthritis,IHD,diabetes,depression,two_conditions,more_than_two_conditions
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.1066553,0.1066553,0,0,0.6824884,0,0,3.445398e-42,1.701963e-36,3.201754e-27,0.02618409,5.043639e-25,0,0,0,8.388629e-168,0,0,1.680867e-28,0,0


In [94]:
primary_care_physician_comparisons_averages <-
  comparator_averages(yearly_calculations, "most_common_primary_care_physician", integration_threshold = 0.75, independece_threshold = 0.75)

primary_care_physician_comparisons_test <-
  comparator_tests(yearly_calculations, "most_common_primary_care_physician", integration_threshold = 0.75, independece_threshold = 0.75)

comparison_results_primary_care_physician <- rbind(
  primary_care_physician_comparisons_averages,
  primary_care_physician_comparisons_test,
  fill = TRUE
)

integration_status,NA
n,NA
N_distinct_patients,NA
N_distinct_specialist,NA
Male,0.1066553
Female,0.1066553
Asian,0
Black,0
Hispanic,0.6824884
North_American_Native,0
White,0


In [98]:
comparison_results_primary_care_physician

integration_status,n,N_distinct_patients,N_distinct_specialist,Male,Female,Asian,Black,Hispanic,North_American_Native,White,Other_race,avg_age,age_65_74,age_75_84,age_85_plus,rural_prp,urban_prp,hypertension,arthritis,IHD,diabetes,depression,two_conditions,more_than_two_conditions,primary_care_physician_integrated,most_common_physician_integrated
<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Independent most_common_primary_care_physician,10396628,1608013,163067,0.4385346,0.5614654,0.01773816,0.06799195,0.01273192,0.002804852,0.8750845,1.363721e-02,7.724706e+01,4.337127e-01,0.34294167,2.233456e-01,0.1863806,0.8136194,0.5778139,3.836148e-02,0.2252955,0.2934294,3.743368e-02,0.2459809,0.08133974,0,0.02889014
Integrated most_common_primary_care_physician,1038379,240950,37891,0.4377101,0.5622899,0.01169611,0.08849659,0.01268419,0.021173387,0.8438971,1.202066e-02,7.713830e+01,4.392259e-01,0.34185495,2.189191e-01,0.2548356,0.7451644,0.4530090,3.293691e-02,0.1919617,0.2534720,3.960307e-02,0.1930095,0.05153513,1,0.76565782
Missing most_common_primary_care_physician,6702662,1114562,8659,0.4348774,0.5651226,0.01350046,0.08460997,0.01683063,0.004971607,0.8617081,1.039274e-02,7.964111e+01,3.503938e-01,0.30562559,3.439806e-01,0.2848762,0.7151238,0.2783466,1.365278e-02,0.1336809,0.1624280,3.633512e-02,0.1141590,0.03012639,0,0.06076705
NA,NA,NA,NA,0.1066553,0.1066553,0.00000000,0.00000000,0.68248840,0.000000000,0.0000000,3.445398e-42,1.701963e-36,3.201754e-27,0.02618409,5.043639e-25,0.0000000,0.0000000,0.0000000,8.388629e-168,0.0000000,0.0000000,1.680867e-28,0.0000000,0.00000000,NA,NA


In [100]:
t(comparison_results_primary_care_physician)

integration_status,Independent most_common_primary_care_physician,Integrated most_common_primary_care_physician,Missing most_common_primary_care_physician,NA
n,10396628,1038379,6702662,NA
N_distinct_patients,1608013,240950,1114562,NA
N_distinct_specialist,163067,37891,8659,NA
Male,0.4385346,0.4377101,0.4348774,0.1066553
Female,0.5614654,0.5622899,0.5651226,0.1066553
Asian,0.01773816,0.01169611,0.01350046,0.00000000
Black,0.06799195,0.08849659,0.08460997,0.00000000
Hispanic,0.01273192,0.01268419,0.01683063,0.68248840
North_American_Native,0.002804852,0.021173387,0.004971607,0.000000000
White,0.8750845,0.8438971,0.8617081,0.0000000


# Overall results

In [106]:
overall_averages <- function(data, specialty, integration_threshold = 0.75, independece_threshold = 0.75) {
  require(tidyverse)
  require(dtplyr)
  varname <- paste(specialty, "_in_facility_non_exclusive_HCPCS_prp", sep = "")
  looking_for <- paste(specialty, "_PRF_PHYSN_NPI", sep = "")


  result <- data %>%
    summarise(
      n = n(),
      N_distinct_patients = length(unique(DESY_SORT_KEY)),
      N_distinct_specialist = length(unique(!!rlang::sym(looking_for))),
      Male = mean(sex == "Male", na.rm = T),
      Female = mean(sex == "Female", na.rm = T),
      Asian = mean(race == "Asian", na.rm = T),
      Black = mean(race == "Black", na.rm = T),
      Hispanic = mean(race == "Hispanic", na.rm = T),
      North_American_Native = mean(race == "North American Native", na.rm = T),
      White = mean(race == "White", na.rm = T),
      Other_race = mean(race == "Other", na.rm = T),
      avg_age = mean(AGE, na.rm = T),
      age_65_74 = mean(na.rm = T, age_group == "65-74"),
      age_75_84 = mean(na.rm = T, age_group == "75-84"),
      age_85_plus = mean(na.rm = T, age_group == "85+"),

      # West=mean(na.rm=T,census_region=="West"),
      # South=mean(na.rm=T,census_region=="South"),
      # Northeast=mean(na.rm=T,census_region=="Northeast"),
      # Midwest=mean(na.rm=T,census_region=="Midwest"),

      rural_prp = mean(RUCC_2013 > 3, na.rm = T),
      urban_prp = mean(RUCC_2013 <= 3, na.rm = T),
      hypertension = mean(hypertension, na.rm = T),
      arthritis = mean(arthritis, na.rm = T),
      IHD = mean(IHD, na.rm = T),
      diabetes = mean(diabetes, na.rm = T),
      depression = mean(depression, na.rm = T),
      two_conditions = mean(number_of_conditions == 2, na.rm = T),
      more_than_two_conditions = mean(number_of_conditions > 2, na.rm = T),
      primary_care_physician_integrated =
        sum(most_common_primary_care_physician_in_facility_non_exclusive_HCPCS_prp >= integration_threshold, na.rm = T) / n(),
      most_common_physician_integrated =
        sum(most_common_physician_in_facility_non_exclusive_HCPCS_prp >= integration_threshold, na.rm = T) / n()
    ) %>%
    as.data.table()

  return(result)
}

In [107]:
primary_care_physician_overall_averages <-
  overall_averages(yearly_calculations, "most_common_primary_care_physician")

In [108]:
t(primary_care_physician_overall_averages)

n,1.813767e+07
N_distinct_patients,2.109378e+06
N_distinct_specialist,1.914590e+05
Male,4.371359e-01
Female,5.628641e-01
Asian,1.582623e-02
Black,7.530692e-02
Hispanic,1.424384e-02
North_American_Native,4.657159e-03
White,8.683559e-01
Other_race,1.234569e-02


# Finding changes in physician patient pools

In [25]:
yearly_calculations <- read_fst("yearly_calculations_cleaned.fst", as.data.table = T)
physician_integration_stats <- read.fst("./physician_integration_stats_before_join.fst", as.data.table = T)

In [26]:
head(yearly_calculations)
head(physician_integration_stats)

DESY_SORT_KEY,year,distinct_clinicians,distinct_primary_care_physicians,hypertension,arthritis,IHD,diabetes,depression,icd_9_pure,icd_10_pure,REFERENCE_YEAR,STATE_CODE,COUNTY_CODE,SEX_CODE,RACE_CODE,AGE,ORIG_REASON_FOR_ENTITLEMENT,CURR_REASON_FOR_ENTITLEMENT,ENTITLEMENT_BUY_IN_IND01,ENTITLEMENT_BUY_IN_IND02,ENTITLEMENT_BUY_IN_IND03,ENTITLEMENT_BUY_IN_IND04,ENTITLEMENT_BUY_IN_IND05,ENTITLEMENT_BUY_IN_IND06,ENTITLEMENT_BUY_IN_IND07,ENTITLEMENT_BUY_IN_IND08,ENTITLEMENT_BUY_IN_IND09,ENTITLEMENT_BUY_IN_IND10,ENTITLEMENT_BUY_IN_IND11,ENTITLEMENT_BUY_IN_IND12,HMO_INDICATOR01,HMO_INDICATOR02,HMO_INDICATOR03,HMO_INDICATOR04,HMO_INDICATOR05,HMO_INDICATOR06,HMO_INDICATOR07,HMO_INDICATOR08,HMO_INDICATOR09,HMO_INDICATOR10,HMO_INDICATOR11,HMO_INDICATOR12,VALID_DATE_OF_DEATH_SWITCH,DATE_OF_DEATH,date_died,date_died_valid,year_of_death,most_common_physician_PRF_PHYSN_NPI,most_common_physician_n,most_common_physician_PRVDR_SPCLTY,most_common_primary_care_physician_PRF_PHYSN_NPI,most_common_primary_care_physician_n,most_common_primary_care_physician_PRVDR_SPCLTY,most_common_physician_in_facility_visits_count,most_common_physician_in_all_visits_count,most_common_physician_in_facility_non_exclusive_HCPCS_count,most_common_physician_in_all_non_exclusive_HCPCS_count,most_common_physician_in_facility_count,most_common_physician_in_all_count,most_common_physician_in_facility_visits_prp,most_common_physician_in_facility_non_exclusive_HCPCS_prp,most_common_physician_in_facility_prp,most_common_primary_care_physician_in_facility_visits_count,most_common_primary_care_physician_in_all_visits_count,most_common_primary_care_physician_in_facility_non_exclusive_HCPCS_count,most_common_primary_care_physician_in_all_non_exclusive_HCPCS_count,most_common_primary_care_physician_in_facility_count,most_common_primary_care_physician_in_all_count,most_common_primary_care_physician_in_facility_visits_prp,most_common_primary_care_physician_in_facility_non_exclusive_HCPCS_prp,most_common_primary_care_physician_in_facility_prp,RUCC_2013,race,sex,age_group,urban,HMO_INDICATOR_sum,ENTITLEMENT_BUY_IN_IND_sum,months_alive,number_of_conditions
<int>,<dbl>,<int>,<int>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<dbl>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<lgl>,<int>,<int>,<dbl>,<int>
100000015,2013,7,1,FALSE,FALSE,TRUE,FALSE,FALSE,TRUE,FALSE,13,22,160,1,1,76,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA,1285600932,2,11,1285600932,2,11,7,96,62,242,66,341,0.07291667,0.2561983,0.19354839,7,96,62,242,66,341,0.07291667,0.2561983,0.19354839,1,White,Male,75-84,TRUE,12,12,NA,1
100000015,2014,10,1,FALSE,FALSE,TRUE,FALSE,FALSE,TRUE,FALSE,14,22,160,1,1,77,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA,1770514119,4,11,1770514119,4,11,16,16,22,22,27,27,1.00000000,1.0000000,1.00000000,16,16,22,22,27,27,1.00000000,1.0000000,1.00000000,1,White,Male,75-84,TRUE,12,12,NA,1
100000015,2015,6,2,TRUE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,15,22,160,1,1,78,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA,1770514119,4,11,1770514119,4,11,48,48,57,57,57,57,1.00000000,1.0000000,1.00000000,48,48,57,57,57,57,1.00000000,1.0000000,1.00000000,1,White,Male,75-84,TRUE,12,12,NA,2
100000015,2016,9,2,TRUE,FALSE,TRUE,TRUE,FALSE,FALSE,TRUE,2016,22,160,1,1,79,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA,1730170630,4,06,1730170630,4,11,0,21,0,21,4,41,0.00000000,0.0000000,0.09756098,0,21,0,21,4,41,0.00000000,0.0000000,0.09756098,1,White,Male,75-84,TRUE,12,12,NA,3
100000015,2017,14,3,FALSE,FALSE,TRUE,TRUE,FALSE,FALSE,TRUE,2017,22,160,1,1,80,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA,

PRF_PHYSN_NPI,year,in_facility_visits_count,in_all_visits_count,in_facility_non_exclusive_HCPCS_count,in_all_non_exclusive_HCPCS_count,in_facility_count,in_all_count,in_facility_visits_prp,in_facility_non_exclusive_HCPCS_prp,in_facility_prp
<chr>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>
,2013,0,35,0,79,0,135,0,0.00000000,0.00000000
,2014,0,12,0,18,1,69,0,0.00000000,0.01449275
,2015,0,13,1,36,1,47,0,0.02777778,0.02127660
,2016,0,9,1,12,1,39,0,0.08333333,0.02564103
,2017,0,2,3,19,3,35,0,0.15789474,0.08571429
,2018,0,3,1,12,1,26,0,0.08333333,0.03846154


In [27]:
physician_integration_change_finder <- function(data, integration_threshold = 0.75, independece_threshold = 0.75) {
  require(tidyverse)
  require(dtplyr)

  result <- data %>%
    mutate(
      integrated = in_facility_non_exclusive_HCPCS_prp >= integration_threshold
    ) %>%
    group_by(PRF_PHYSN_NPI) %>%
    mutate(
      integration_change =
        case_when(
          integrated == T & lag(integrated) == F ~ "newly_integrated",
          integrated == F & lag(integrated) == T ~ "newly_disintegrated",
          is.na(integrated) & lag(integrated) == F ~ "remained_independent",
          integrated == F & lag(integrated) == F ~ "remained_independent",
          integrated == T & lag(integrated) == T ~ "remained_integrated",
          integrated == T & is.na(lag(integrated)) & lag(integrated, 2) == T ~ "remained_integrated",
          integrated == T & is.na(lag(integrated)) & lag(integrated, 2) == F ~ "newly_integrated",
          is.na(integrated) & lag(integrated) == T ~ "remained_integrated",
          year == 2013 & integrated == T ~ "remained_integrated",
          year == 2013 & integrated == F ~ "remained_independent"
        ),
    ) %>%
    group_by(PRF_PHYSN_NPI) %>%
    mutate(
      sum_integration_changes = sum(integration_change %in% c("newly_integrated", "newly_disintegrated"), na.rm = T),
      integrated_once_and_stayed = sum(integration_change == "newly_disintegrated", na.rm = T) == 0 & sum(integration_change == "newly_integrated", na.rm = T) == 1,
    ) %>%
    group_by(PRF_PHYSN_NPI) %>%
    mutate(
      year_integrated = case_when(integration_change == "newly_integrated" & integrated_once_and_stayed == T ~ year)
    ) %>%
    group_by(PRF_PHYSN_NPI) %>%
    mutate(
      year_integrated = year_integrated[which(!is.na(year_integrated))]
    ) %>%
    mutate(years_from_integration = year - year_integrated) %>%
    as.data.table()
}

In [28]:
physician_integration_stats_changes <- physician_integration_change_finder(physician_integration_stats)

In [29]:
yearly_calculations <- left_join(
  yearly_calculations,
  physician_integration_stats_changes[, .(PRF_PHYSN_NPI, year, integrated, integration_change, sum_integration_changes, integrated_once_and_stayed, year_integrated, years_from_integration)],
  by = c("most_common_primary_care_physician_PRF_PHYSN_NPI" = "PRF_PHYSN_NPI", "year" = "year")
) %>% as.data.table()

In [30]:
rename_last <- function(data, how_many, new_names) {
  total_cols <- ncol(data)
  setnames(data, (total_cols - how_many + 1):(total_cols), new_names)
}

rename_last(yearly_calculations, 6, paste("most_common_primary_care_physician_", colnames(yearly_calculations[, (ncol(yearly_calculations) - 5):ncol(yearly_calculations)]), sep = ""))

In [31]:
yearly_calculations <- yearly_calculations %>%
  group_by(DESY_SORT_KEY) %>%
  mutate(
    most_common_primary_care_physician_changed = case_when(
      most_common_primary_care_physician_PRF_PHYSN_NPI != lag(most_common_primary_care_physician_PRF_PHYSN_NPI) ~ T,
      most_common_primary_care_physician_PRF_PHYSN_NPI == lag(most_common_primary_care_physician_PRF_PHYSN_NPI) ~ F,
      year == 2013 ~ F
    )
  ) %>%
  as.data.table()

In [32]:
write.fst(yearly_calculations, "yearly_calculations_with_integration_changes.fst")

## Summarizing

### Reading the data with integration changes

In [3]:
yearly_calculations <- read_fst("yearly_calculations_with_integration_changes.fst", as.data.table = T)
head(yearly_calculations)

DESY_SORT_KEY,year,distinct_clinicians,distinct_primary_care_physicians,hypertension,arthritis,IHD,diabetes,depression,icd_9_pure,icd_10_pure,REFERENCE_YEAR,STATE_CODE,COUNTY_CODE,SEX_CODE,RACE_CODE,AGE,ORIG_REASON_FOR_ENTITLEMENT,CURR_REASON_FOR_ENTITLEMENT,ENTITLEMENT_BUY_IN_IND01,ENTITLEMENT_BUY_IN_IND02,ENTITLEMENT_BUY_IN_IND03,ENTITLEMENT_BUY_IN_IND04,ENTITLEMENT_BUY_IN_IND05,ENTITLEMENT_BUY_IN_IND06,ENTITLEMENT_BUY_IN_IND07,ENTITLEMENT_BUY_IN_IND08,ENTITLEMENT_BUY_IN_IND09,ENTITLEMENT_BUY_IN_IND10,ENTITLEMENT_BUY_IN_IND11,ENTITLEMENT_BUY_IN_IND12,HMO_INDICATOR01,HMO_INDICATOR02,HMO_INDICATOR03,HMO_INDICATOR04,HMO_INDICATOR05,HMO_INDICATOR06,HMO_INDICATOR07,HMO_INDICATOR08,HMO_INDICATOR09,HMO_INDICATOR10,HMO_INDICATOR11,HMO_INDICATOR12,VALID_DATE_OF_DEATH_SWITCH,DATE_OF_DEATH,date_died,date_died_valid,year_of_death,most_common_physician_PRF_PHYSN_NPI,most_common_physician_n,most_common_physician_PRVDR_SPCLTY,most_common_primary_care_physician_PRF_PHYSN_NPI,most_common_primary_care_physician_n,most_common_primary_care_physician_PRVDR_SPCLTY,most_common_physician_in_facility_visits_count,most_common_physician_in_all_visits_count,most_common_physician_in_facility_non_exclusive_HCPCS_count,most_common_physician_in_all_non_exclusive_HCPCS_count,most_common_physician_in_facility_count,most_common_physician_in_all_count,most_common_physician_in_facility_visits_prp,most_common_physician_in_facility_non_exclusive_HCPCS_prp,most_common_physician_in_facility_prp,most_common_primary_care_physician_in_facility_visits_count,most_common_primary_care_physician_in_all_visits_count,most_common_primary_care_physician_in_facility_non_exclusive_HCPCS_count,most_common_primary_care_physician_in_all_non_exclusive_HCPCS_count,most_common_primary_care_physician_in_facility_count,most_common_primary_care_physician_in_all_count,most_common_primary_care_physician_in_facility_visits_prp,most_common_primary_care_physician_in_facility_non_exclusive_HCPCS_prp,most_common_primary_care_physician_in_facility_prp,RUCC_2013,race,sex,age_group,urban,HMO_INDICATOR_sum,ENTITLEMENT_BUY_IN_IND_sum,months_alive,number_of_conditions,most_common_primary_care_physician_integrated,most_common_primary_care_physician_integration_change,most_common_primary_care_physician_sum_integration_changes,most_common_primary_care_physician_integrated_once_and_stayed,most_common_primary_care_physician_year_integrated,most_common_primary_care_physician_years_from_integration,most_common_primary_care_physician_changed
<int>,<dbl>,<int>,<int>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<dbl>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<lgl>,<int>,<int>,<dbl>,<int>,<lgl>,<chr>,<int>,<lgl>,<dbl>,<dbl>,<lgl>
100000015,2013,7,1,FALSE,FALSE,TRUE,FALSE,FALSE,TRUE,FALSE,13,22,160,1,1,76,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA,1285600932,2,11,1285600932,2,11,7,96,62,242,66,341,0.07291667,0.2561983,0.19354839,7,96,62,242,66,341,0.07291667,0.2561983,0.19354839,1,White,Male,75-84,TRUE,12,12,NA,1,FALSE,remained_independent,0,FALSE,NA,NA,FALSE
100000015,2014,10,1,FALSE,FALSE,TRUE,FALSE,FALSE,TRUE,FALSE,14,22,160,1,1,77,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA,1770514119,4,11,1770514119,4,11,16,16,22,22,27,27,1.00000000,1.0000000,1.00000000,16,16,22,22,27,27,1.00000000,1.0000000,1.00000000,1,White,Male,75-84,TRUE,12,12,NA,1,TRUE,remained_integrated,0,FALSE,NA,NA,TRUE
100000015,2015,6,2,TRUE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,15,22,160,1,1,78,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA,1770514119,4,11,1770514119,4,11,48,48,57,57,57,57,1.00000000,1.0000000,1.00000000,48,48,57,57,57,57,1.00000000

### Summary function

In [4]:
integration_change_summarizer <- function(data) {
  require(tidyverse)
  require(dtplyr)

  result <-
    data %>%
    filter(!is.na(most_common_primary_care_physician_years_from_integration)) %>%
    group_by(most_common_primary_care_physician_years_from_integration) %>%
    summarise(
      hypertension = mean(hypertension, na.rm = T),
      arthritis = mean(arthritis, na.rm = T),
      IHD = mean(IHD, na.rm = T),
      diabetes = mean(diabetes, na.rm = T),
      depression = mean(depression, na.rm = T),
      two_conditions = mean(number_of_conditions == 2, na.rm = T),
      more_than_two_conditions = mean(number_of_conditions > 2, na.rm = T)
    ) %>%
    as.data.table()
}

In [ ]:
integration_changes_years_to_integration <- integration_change_summarizer(yearly_calculations)

In [ ]:
integration_changes_years_to_integration

most_common_primary_care_physician_years_from_integration,hypertension,arthritis,IHD,diabetes,depression,two_conditions,more_than_two_conditions
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
-7,0.5447631,0.02607562,0.2114298,0.2588005,0.01955671,0.2244676,0.04997827
-6,0.5379679,0.02631016,0.2011765,0.2684492,0.01871658,0.2186096,0.05262032
-5,0.5410039,0.02768023,0.2003030,0.2673001,0.02230944,0.2186876,0.05570474
-4,0.5505279,0.03580986,0.1959811,0.2626867,0.02865762,0.2244441,0.05760716
-3,0.5459895,0.03590705,0.1985757,0.2674288,0.03159670,0.2240630,0.06034483
-2,0.5367955,0.04013226,0.1942608,0.2654857,0.03552145,0.2232907,0.05939453
-1,0.5219994,0.03901461,0.1847035,0.2594672,0.03703810,0.2133773,0.05508450
0,0.4856682,0.04088428,0.1735608,0.2481892,0.03985445,0.1935052,0.05118259
1,0.4797038,0.04223807,0.1723219,0.2432803,0.04290416,0.1912859,0.05015281


In [53]:
ggplot_all_variables <- ggplot(data = integration_changes_years_to_integration) +
  geom_vline(xintercept = 0, linetype = "dotted", color = "black", size = 1.5) +
  geom_line(aes(x = most_common_primary_care_physician_years_from_integration, y = hypertension, color = "hypertension")) +
  geom_point(aes(x = most_common_primary_care_physician_years_from_integration, y = hypertension, color = "hypertension")) +
  geom_line(aes(x = most_common_primary_care_physician_years_from_integration, y = arthritis, color = "arthritis")) +
  geom_point(aes(x = most_common_primary_care_physician_years_from_integration, y = arthritis, color = "arthritis")) +
  geom_line(aes(x = most_common_primary_care_physician_years_from_integration, y = IHD, color = "IHD")) +
  geom_point(aes(x = most_common_primary_care_physician_years_from_integration, y = IHD, color = "IHD")) +
  geom_line(aes(x = most_common_primary_care_physician_years_from_integration, y = diabetes, color = "diabetes")) +
  geom_point(aes(x = most_common_primary_care_physician_years_from_integration, y = diabetes, color = "diabetes")) +
  geom_line(aes(x = most_common_primary_care_physician_years_from_integration, y = depression, color = "depression")) +
  geom_point(aes(x = most_common_primary_care_physician_years_from_integration, y = depression, color = "depression")) +
  geom_line(aes(x = most_common_primary_care_physician_years_from_integration, y = two_conditions, color = "two_conditions")) +
  geom_point(aes(x = most_common_primary_care_physician_years_from_integration, y = two_conditions, color = "two_conditions")) +
  geom_line(aes(x = most_common_primary_care_physician_years_from_integration, y = more_than_two_conditions, color = "more_than_two_conditions")) +
  geom_point(aes(x = most_common_primary_care_physician_years_from_integration, y = more_than_two_conditions, color = "more_than_two_conditions")) +
  scale_colour_discrete() +
  scale_x_continuous(breaks = seq(-8, 8, 2)) +
  xlab("Primary care physician years from integration") +
  ylab("Prevalence")

ggsave(ggplot_all_variables, filename = "ggplot_all_variables.pdf", width = 12, height = 7)

In [54]:
ggplot_arthritis_depression_more_than_two <- ggplot(data = integration_changes_years_to_integration) +
  geom_vline(xintercept = 0, linetype = "dotted", color = "black", size = 1.5) +
  geom_line(aes(x = most_common_primary_care_physician_years_from_integration, y = arthritis, color = "arthritis")) +
  geom_point(aes(x = most_common_primary_care_physician_years_from_integration, y = arthritis, color = "arthritis")) +
  geom_line(aes(x = most_common_primary_care_physician_years_from_integration, y = depression, color = "depression")) +
  geom_point(aes(x = most_common_primary_care_physician_years_from_integration, y = depression, color = "depression")) +
  geom_line(aes(x = most_common_primary_care_physician_years_from_integration, y = more_than_two_conditions, color = "more_than_two_conditions")) +
  geom_point(aes(x = most_common_primary_care_physician_years_from_integration, y = more_than_two_conditions, color = "more_than_two_conditions")) +
  scale_colour_discrete() +
  scale_x_continuous(breaks = seq(-8, 8, 2)) +
  xlab("Primary care physician years from integration") +
  ylab("Prevalence")

ggsave(ggplot_arthritis_depression_more_than_two, filename = "ggplot_arthritis_depression_more_than_two.pdf", width = 12, height = 7)

In [59]:
ggplot_IHD_diabetes_two_conditions <- ggplot(data = integration_changes_years_to_integration) +
  geom_vline(xintercept = 0, linetype = "dotted", color = "black", size = 1.5) +
  geom_line(aes(x = most_common_primary_care_physician_years_from_integration, y = IHD, color = "IHD")) +
  geom_point(aes(x = most_common_primary_care_physician_years_from_integration, y = IHD, color = "IHD")) +
  geom_line(aes(x = most_common_primary_care_physician_years_from_integration, y = diabetes, color = "diabetes")) +
  geom_point(aes(x = most_common_primary_care_physician_years_from_integration, y = diabetes, color = "diabetes")) +
  geom_line(aes(x = most_common_primary_care_physician_years_from_integration, y = two_conditions, color = "two_conditions")) +
  geom_point(aes(x = most_common_primary_care_physician_years_from_integration, y = two_conditions, color = "two_conditions")) +
  scale_colour_discrete() +
  scale_x_continuous(breaks = seq(-8, 8, 2)) +
  xlab("Primary care physician years from integration") +
  ylab("Prevalence")

ggsave(ggplot_IHD_diabetes_two_conditions, filename = "ggplot_IHD_diabetes_two_conditions.pdf", width = 12, height = 7)

In [100]:
ggplot_hypertension_rdd <- ggplot(data = integration_changes_years_to_integration %>% mutate(before = (most_common_primary_care_physician_years_from_integration < 0)) %>% as.data.frame()) +
  geom_vline(xintercept = 0, linetype = "dotted", color = "black", size = 1.5) +
  geom_line(aes(x = most_common_primary_care_physician_years_from_integration, y = hypertension, color = "hypertension")) +
  geom_point(aes(x = most_common_primary_care_physician_years_from_integration, y = hypertension, color = "hypertension")) +
  scale_colour_discrete() +
  scale_x_continuous(breaks = seq(-8, 8, 2)) +
  xlab("Primary care physician years from integration") +
  ylab("Prevalence") +
  geom_smooth(method = "lm", aes(x = most_common_primary_care_physician_years_from_integration, y = hypertension, color = "hypertension", fill = "hypertension", group = before))



ggsave(ggplot_hypertension_rdd, filename = "ggplot_hypertension_rdd.pdf", width = 12, height = 7)

`geom_smooth()` using formula 'y ~ x'



In [101]:
ggplot_IHD_diabetes_two_conditions_rdd <- ggplot(data = integration_changes_years_to_integration %>% mutate(before = (most_common_primary_care_physician_years_from_integration < 0)) %>% as.data.frame()) +
  geom_vline(xintercept = 0, linetype = "dotted", color = "black", size = 1.5) +
  geom_line(aes(x = most_common_primary_care_physician_years_from_integration, y = IHD, color = "IHD")) +
  geom_point(aes(x = most_common_primary_care_physician_years_from_integration, y = IHD, color = "IHD")) +
  geom_line(aes(x = most_common_primary_care_physician_years_from_integration, y = diabetes, color = "diabetes")) +
  geom_point(aes(x = most_common_primary_care_physician_years_from_integration, y = diabetes, color = "diabetes")) +
  geom_line(aes(x = most_common_primary_care_physician_years_from_integration, y = two_conditions, color = "two_conditions")) +
  geom_point(aes(x = most_common_primary_care_physician_years_from_integration, y = two_conditions, color = "two_conditions")) +
  scale_colour_discrete() +
  scale_x_continuous(breaks = seq(-8, 8, 2)) +
  xlab("Primary care physician years from integration") +
  ylab("Prevalence") +
  geom_smooth(method = "lm", aes(x = most_common_primary_care_physician_years_from_integration, y = IHD, color = "IHD", fill = "IHD", group = before)) +
  geom_smooth(method = "lm", aes(x = most_common_primary_care_physician_years_from_integration, y = diabetes, color = "diabetes", fill = "diabetes", group = before)) +
  geom_smooth(method = "lm", aes(x = most_common_primary_care_physician_years_from_integration, y = two_conditions, color = "two_conditions", fill = "two_conditions", group = before))



ggsave(ggplot_IHD_diabetes_two_conditions_rdd, filename = "ggplot_IHD_diabetes_two_conditions_rdd.pdf", width = 12, height = 7)

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'



In [102]:
ggplot_arthritis_depression_more_than_two_rdd <-
  ggplot(data = integration_changes_years_to_integration %>% mutate(before = (most_common_primary_care_physician_years_from_integration < 0)) %>% as.data.frame()) +
  geom_vline(xintercept = 0, linetype = "dotted", color = "black", size = 1.5) +
  geom_line(aes(x = most_common_primary_care_physician_years_from_integration, y = arthritis, color = "arthritis")) +
  geom_point(aes(x = most_common_primary_care_physician_years_from_integration, y = arthritis, color = "arthritis")) +
  geom_line(aes(x = most_common_primary_care_physician_years_from_integration, y = depression, color = "depression")) +
  geom_point(aes(x = most_common_primary_care_physician_years_from_integration, y = depression, color = "depression")) +
  geom_line(aes(x = most_common_primary_care_physician_years_from_integration, y = more_than_two_conditions, color = "more_than_two_conditions")) +
  geom_point(aes(x = most_common_primary_care_physician_years_from_integration, y = more_than_two_conditions, color = "more_than_two_conditions")) +
  scale_colour_discrete() +
  scale_x_continuous(breaks = seq(-8, 8, 2)) +
  xlab("Primary care physician years from integration") +
  ylab("Prevalence") +
  geom_smooth(method = "lm", aes(x = most_common_primary_care_physician_years_from_integration, y = arthritis, color = "arthritis", fill = "arthritis", group = before)) +
  geom_smooth(method = "lm", aes(x = most_common_primary_care_physician_years_from_integration, y = depression, color = "depression", fill = "depression", group = before)) +
  geom_smooth(method = "lm", aes(x = most_common_primary_care_physician_years_from_integration, y = more_than_two_conditions, color = "more_than_two_conditions", fill = "more_than_two_conditions", group = before))

ggsave(ggplot_arthritis_depression_more_than_two_rdd, filename = "ggplot_arthritis_depression_more_than_two_rdd.pdf", width = 12, height = 7)

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'

